In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#y_airfoil = np.reshape(y_airfoil.values, (-1,1))
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)

In [13]:
def standard_scaler(X_train, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    x_test=X_scale_train.transform(x_test)
    return X_train, x_test, X_scale_train

In [14]:
def min_max_scaler_forest(X_train, x_test, Y_train, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    y_test=Y_scale_train.transform(y_test)
    return X_train, x_test, Y_train, y_test, X_scale_train, Y_scale_train

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_1(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [19]:
def NN_model_structure_regression_2(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [20]:
def NN_model_structure_regression_3(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [21]:
def NN_model_structure_regression_4(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [22]:
def NN_model_structure_regression_5(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [23]:
def NN_model_structure_regression_6(X_train, X_val, Y_train, Y_val, adam, size, epochs):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    return model, history

In [25]:
x_train_airfoil, x_test_airfoil, x_scale_train_airfoil=standard_scaler(x_train_airfoil, x_test_airfoil)
#_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil, x_scale_train_airfoil, y_scale_train_airfoil=min_max_scaler_forest(x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil)
x_train_forest, x_test_forest, y_train_forest, y_test_forest, x_scale_train_forest, y_scale_train_forest=min_max_scaler_forest(x_train_forest, x_test_forest, y_train_forest, y_test_forest)
x_train_housing, x_test_housing, X_scale_train_housing=standard_scaler(x_train_housing, x_test_housing)

In [26]:
model_airfoil, history_airfoil=NN_model_structure_regression_1(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 420us/step - lo

1052/1052 [==============================] - ETA: 0s - loss: 22.59 - 0s 99us/step - loss: 22.3230 - val_loss: 25.4991
Epoch 66/1000
1052/1052 [==============================] - 0s 94us/step - loss: 21.8059 - val_loss: 26.6292
Epoch 67/1000
1052/1052 [==============================] - 0s 107us/step - loss: 22.0500 - val_loss: 25.9013
Epoch 68/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.8269 - val_loss: 26.8217
Epoch 69/1000
1052/1052 [==============================] - 0s 97us/step - loss: 22.0763 - val_loss: 25.3539
Epoch 70/1000
1052/1052 [==============================] - 0s 98us/step - loss: 21.6827 - val_loss: 25.7999
Epoch 71/1000
1052/1052 [==============================] - 0s 97us/step - loss: 22.1656 - val_loss: 25.7877
Epoch 72/1000
1052/1052 [==============================] - 0s 98us/step - loss: 22.2649 - val_loss: 25.8903
Epoch 73/1000
1052/1052 [==============================] - 0s 95us/step - loss: 22.0316 - val_loss: 26.1558
Epoch 74/1000
105

1052/1052 [==============================] - 0s 95us/step - loss: 17.4034 - val_loss: 21.0010
Epoch 216/1000
1052/1052 [==============================] - 0s 93us/step - loss: 16.9459 - val_loss: 19.6376
Epoch 217/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.1787 - val_loss: 19.9228
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 17.3325 - val_loss: 20.8439
Epoch 219/1000
1052/1052 [==============================] - 0s 92us/step - loss: 16.7546 - val_loss: 22.3299
Epoch 220/1000
1052/1052 [==============================] - 0s 99us/step - loss: 17.6438 - val_loss: 19.8830
Epoch 221/1000
1052/1052 [==============================] - 0s 91us/step - loss: 16.5753 - val_loss: 19.1888
Epoch 222/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.7364 - val_loss: 18.7288
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.6339 - val_loss: 19.0879
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 98us/step - loss: 9.1916 - val_loss: 11.4398
Epoch 366/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.1102 - val_loss: 11.7826
Epoch 367/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2983 - val_loss: 13.6353
Epoch 368/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.0482 - val_loss: 11.2827
Epoch 369/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.4119 - val_loss: 12.6844
Epoch 370/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.6829 - val_loss: 11.6362
Epoch 371/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.8859 - val_loss: 11.3968
Epoch 372/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.2511 - val_loss: 11.0712
Epoch 373/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.8979 - val_loss: 11.0069
Epoch 374/1000
1052/1052 [===============

Epoch 441/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4543 - val_loss: 10.8837
Epoch 442/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5488 - val_loss: 10.8761
Epoch 443/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.5261 - val_loss: 11.0741
Epoch 444/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.4758 - val_loss: 10.4154
Epoch 445/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5721 - val_loss: 11.3610
Epoch 446/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9611 - val_loss: 10.0119
Epoch 447/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6824 - val_loss: 12.2901
Epoch 448/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7255 - val_loss: 10.2684
Epoch 449/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8390 - val_loss: 10.7644
Epoch 450/1000
1052/1052 [==

Epoch 517/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1945 - val_loss: 10.5516
Epoch 518/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3383 - val_loss: 10.7151
Epoch 519/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.5788 - val_loss: 10.2698
Epoch 520/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5592 - val_loss: 9.8783
Epoch 521/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4627 - val_loss: 10.7476
Epoch 522/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4425 - val_loss: 10.0406
Epoch 523/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8826 - val_loss: 10.0009
Epoch 524/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.6022 - val_loss: 10.4154
Epoch 525/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6490 - val_loss: 9.7367
Epoch 526/1000
1052/1052 [====

1052/1052 [==============================] - 0s 90us/step - loss: 8.2584 - val_loss: 9.5629
Epoch 594/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2523 - val_loss: 11.5094
Epoch 595/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.3405 - val_loss: 9.6602
Epoch 596/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1999 - val_loss: 11.2769
Epoch 597/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.3859 - val_loss: 10.5817
Epoch 598/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6766 - val_loss: 9.8540
Epoch 599/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9456 - val_loss: 10.4575
Epoch 600/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3046 - val_loss: 10.1441
Epoch 601/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6811 - val_loss: 10.2384
Epoch 602/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 89us/step - loss: 9.0458 - val_loss: 9.6115
Epoch 746/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.5742 - val_loss: 9.4098
Epoch 747/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1140 - val_loss: 9.4041
Epoch 748/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3337 - val_loss: 12.1390
Epoch 749/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.3147 - val_loss: 9.4832
Epoch 750/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.9935 - val_loss: 9.4288
Epoch 751/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3496 - val_loss: 9.5936
Epoch 752/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.0852 - val_loss: 10.0055
Epoch 753/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.0012 - val_loss: 9.6810
Epoch 754/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 88us/step - loss: 8.0425 - val_loss: 9.1376
Epoch 898/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1817 - val_loss: 9.3813
Epoch 899/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5021 - val_loss: 9.1349
Epoch 900/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1693 - val_loss: 9.4929
Epoch 901/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0874 - val_loss: 9.7685
Epoch 902/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1316 - val_loss: 9.0599
Epoch 903/1000
1052/1052 [==============================] - 0s 105us/step - loss: 8.0588 - val_loss: 9.5443
Epoch 904/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.1139 - val_loss: 9.9959
Epoch 905/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.1310 - val_loss: 9.2721
Epoch 906/1000
1052/1052 [========================

[array([[-1.8719815 , -0.70353127, -0.16644102, -3.2435915 ,  3.8360088 ],
        [ 0.48370406, -1.4035313 , -0.18981631,  0.09060076, -0.21139403],
        [ 0.4033324 , -1.5788172 , -0.28289875, -0.00454685,  0.7078572 ],
        [ 0.02927194,  0.25109854,  0.1316003 ,  0.05463107, -0.1432545 ],
        [-0.32422116,  0.29048452, -0.18447773, -2.4228208 ,  0.328253  ]],
       dtype=float32),
 array([-1.2950469, -0.7019617,  1.530619 , -4.7311363,  4.774982 ],
       dtype=float32),
 array([[ 1.008399  , -0.21073602,  0.88689995, -0.7137894 , -1.2686211 ],
        [ 1.0261103 ,  0.00938091,  0.88590646, -0.49040246,  0.12281685],
        [ 2.8317869 , -3.0264223 ,  2.5568082 , -2.7092123 , -3.0915775 ],
        [-2.7250457 ,  1.6903435 , -2.6647012 ,  2.1479414 ,  2.1384213 ],
        [ 2.9609253 , -2.2491028 ,  3.0956051 , -3.271654  , -2.986916  ]],
       dtype=float32),
 array([ 2.3604026, -2.4299738,  2.2260025, -2.3108332, -2.3956265],
       dtype=float32),
 array([[ 2.471527

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure1.h5')

In [28]:
model_airfoil, history_airfoil=NN_model_structure_regression_2(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 194us/step - l

1052/1052 [==============================] - 0s 92us/step - loss: 18.1922 - val_loss: 24.3840
Epoch 68/1000
1052/1052 [==============================] - 0s 95us/step - loss: 17.7318 - val_loss: 22.1198
Epoch 69/1000
1052/1052 [==============================] - 0s 98us/step - loss: 17.7113 - val_loss: 21.4852
Epoch 70/1000
1052/1052 [==============================] - 0s 94us/step - loss: 17.5641 - val_loss: 21.3769
Epoch 71/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.7261 - val_loss: 21.6010
Epoch 72/1000
1052/1052 [==============================] - 0s 92us/step - loss: 17.4887 - val_loss: 20.9944
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 17.6381 - val_loss: 21.1731
Epoch 74/1000
1052/1052 [==============================] - 0s 84us/step - loss: 17.8148 - val_loss: 21.1669
Epoch 75/1000
1052/1052 [==============================] - 0s 91us/step - loss: 17.2396 - val_loss: 21.0965
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 88us/step - loss: 11.3784 - val_loss: 13.2675
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.6999 - val_loss: 16.5702
Epoch 219/1000
1052/1052 [==============================] - 0s 93us/step - loss: 11.3149 - val_loss: 12.8785
Epoch 220/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.1636 - val_loss: 13.3541
Epoch 221/1000
1052/1052 [==============================] - 0s 91us/step - loss: 10.9366 - val_loss: 12.9735
Epoch 222/1000
1052/1052 [==============================] - 0s 96us/step - loss: 10.7069 - val_loss: 14.3181
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.2770 - val_loss: 13.1054
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.5570 - val_loss: 13.7235
Epoch 225/1000
1052/1052 [==============================] - 0s 95us/step - loss: 10.4632 - val_loss: 13.0668
Epoch 226/1000
1052/1052 [========

Epoch 368/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4387 - val_loss: 10.7616
Epoch 369/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.7400 - val_loss: 10.2635
Epoch 370/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8496 - val_loss: 11.8528
Epoch 371/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.7406 - val_loss: 10.6022
Epoch 372/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.6181 - val_loss: 10.3138
Epoch 373/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8977 - val_loss: 10.1131
Epoch 374/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.5952 - val_loss: 10.1328
Epoch 375/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4433 - val_loss: 10.5299
Epoch 376/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3142 - val_loss: 14.2315
Epoch 377/1000
1052/1052 [==

Epoch 444/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.4777 - val_loss: 10.0237
Epoch 445/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.6883 - val_loss: 9.7809
Epoch 446/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4367 - val_loss: 10.4180
Epoch 447/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3806 - val_loss: 10.2408
Epoch 448/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.8658 - val_loss: 10.5291
Epoch 449/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.7273 - val_loss: 9.6915
Epoch 450/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6328 - val_loss: 10.9763
Epoch 451/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4780 - val_loss: 10.0483
Epoch 452/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3007 - val_loss: 9.6921
Epoch 453/1000
1052/1052 [=====

1052/1052 [==============================] - 0s 87us/step - loss: 8.2860 - val_loss: 10.9810
Epoch 521/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.0729 - val_loss: 9.6007
Epoch 522/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3439 - val_loss: 9.4525
Epoch 523/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3230 - val_loss: 10.5800
Epoch 524/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.0229 - val_loss: 12.5287
Epoch 525/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4029 - val_loss: 9.7482
Epoch 526/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0984 - val_loss: 12.4106
Epoch 527/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3074 - val_loss: 10.1462
Epoch 528/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.4472 - val_loss: 9.6132
Epoch 529/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 89us/step - loss: 8.1925 - val_loss: 11.8293
Epoch 673/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.4683 - val_loss: 9.8904
Epoch 674/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4462 - val_loss: 9.6691
Epoch 675/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4160 - val_loss: 10.3770
Epoch 676/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4117 - val_loss: 9.7589
Epoch 677/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1670 - val_loss: 9.9016
Epoch 678/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.3287 - val_loss: 9.5697
Epoch 679/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2269 - val_loss: 9.6787
Epoch 680/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.9985 - val_loss: 9.2151
Epoch 681/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 91us/step - loss: 8.7212 - val_loss: 9.2532
Epoch 825/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3675 - val_loss: 10.3145
Epoch 826/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2724 - val_loss: 9.6759
Epoch 827/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3623 - val_loss: 9.4245
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2957 - val_loss: 11.5659
Epoch 829/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4656 - val_loss: 11.1549
Epoch 830/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6683 - val_loss: 9.7876
Epoch 831/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2815 - val_loss: 9.8867
Epoch 832/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2457 - val_loss: 9.1952
Epoch 833/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 90us/step - loss: 8.0465 - val_loss: 8.9928
Epoch 977/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3903 - val_loss: 9.3542
Epoch 978/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.1370 - val_loss: 12.1327
Epoch 979/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2981 - val_loss: 9.0360
Epoch 980/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.0715 - val_loss: 9.0579
Epoch 981/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2762 - val_loss: 10.2796
Epoch 982/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.2880 - val_loss: 10.4505
Epoch 983/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3943 - val_loss: 9.8672
Epoch 984/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.2878 - val_loss: 10.2656
Epoch 985/1000
1052/1052 [======================

[array([[-0.18546297,  0.7904468 , -3.3235636 , -1.8288312 , -4.0985484 ],
        [-0.16851097,  1.5531429 ,  0.09846704,  0.52501965,  0.21355905],
        [-0.29885468,  1.7086195 ,  0.01912374,  0.42133436, -0.73038805],
        [ 0.11626258, -0.2721323 ,  0.05982428,  0.01913126,  0.17582665],
        [-0.17599642, -0.40401977, -2.4137888 , -0.30316097, -0.3241332 ]],
       dtype=float32),
 array([ 1.2750512,  0.6981047, -4.891635 , -1.3011069, -5.030824 ],
       dtype=float32),
 array([[-1.8419304 , -1.5128286 ,  1.5209936 , -1.8290058 ,  1.5132009 ,
         -1.224721  ,  1.4473449 ,  1.5925415 ,  1.4079596 , -0.82628655],
        [ 0.3778976 ,  0.6369942 ,  0.31025425,  0.40834126, -0.4209189 ,
          0.22520089, -0.5094401 , -0.09394934, -0.2854539 ,  0.47769466],
        [ 2.1376326 ,  1.6746633 , -2.3704526 ,  1.3334824 , -1.5715615 ,
          2.2088645 , -2.043277  , -1.6855069 , -1.335016  ,  2.203931  ],
        [-0.52063954, -0.78502256,  1.0060669 , -0.44593796,  

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure2.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_3(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                90        
_________________________________________________________________
activation_6 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 212us/step - l

1052/1052 [==============================] - 0s 100us/step - loss: 16.1449 - val_loss: 18.9845
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.2901 - val_loss: 18.8770
Epoch 69/1000
1052/1052 [==============================] - 0s 83us/step - loss: 16.0071 - val_loss: 20.3350
Epoch 70/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.4295 - val_loss: 19.1770
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.9162 - val_loss: 20.1480
Epoch 72/1000
1052/1052 [==============================] - 0s 93us/step - loss: 16.2334 - val_loss: 19.1652
Epoch 73/1000
1052/1052 [==============================] - 0s 91us/step - loss: 15.7977 - val_loss: 19.4801
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.2564 - val_loss: 18.3747
Epoch 75/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.2504 - val_loss: 19.4634
Epoch 76/1000
1052/1052 [================

1052/1052 [==============================] - 0s 98us/step - loss: 9.7769 - val_loss: 13.3076
Epoch 218/1000
1052/1052 [==============================] - 0s 102us/step - loss: 10.3259 - val_loss: 11.7477
Epoch 219/1000
1052/1052 [==============================] - 0s 103us/step - loss: 9.9939 - val_loss: 12.1894
Epoch 220/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.9577 - val_loss: 11.7005
Epoch 221/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.5709 - val_loss: 11.5609
Epoch 222/1000
1052/1052 [==============================] - 0s 103us/step - loss: 9.4632 - val_loss: 12.3245
Epoch 223/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.6972 - val_loss: 12.0770
Epoch 224/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.7613 - val_loss: 12.5407
Epoch 225/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.7109 - val_loss: 12.1488
Epoch 226/1000
1052/1052 [=============

Epoch 293/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.4254 - val_loss: 11.4166
Epoch 294/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.7734 - val_loss: 14.9185
Epoch 295/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.6127 - val_loss: 10.9819
Epoch 296/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.3019 - val_loss: 11.4079
Epoch 297/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.3741 - val_loss: 11.5974
Epoch 298/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.0845 - val_loss: 12.1620
Epoch 299/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.3250 - val_loss: 11.6837
Epoch 300/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.0481 - val_loss: 13.4119
Epoch 301/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.3682 - val_loss: 11.7626
Epoch 302/1000
1052/1052 [==

Epoch 369/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.4094 - val_loss: 11.7924
Epoch 370/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.3429 - val_loss: 11.7430
Epoch 371/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0191 - val_loss: 12.0111
Epoch 372/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0683 - val_loss: 11.5161
Epoch 373/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0501 - val_loss: 11.6607
Epoch 374/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8901 - val_loss: 11.1681
Epoch 375/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.0141 - val_loss: 11.0007
Epoch 376/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.0050 - val_loss: 10.9485
Epoch 377/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9778 - val_loss: 11.7894
Epoch 378/1000
1052/1052 [==

...

Epoch 673/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.0334 - val_loss: 10.7736
Epoch 674/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0537 - val_loss: 11.0144
Epoch 675/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.9133 - val_loss: 11.2568
Epoch 676/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.3801 - val_loss: 11.3767
Epoch 677/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.3382 - val_loss: 10.8689
Epoch 678/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1795 - val_loss: 11.1917
Epoch 679/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.1103 - val_loss: 11.3584
Epoch 680/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.1646 - val_loss: 11.4151
Epoch 681/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9090 - val_loss: 10.8473
Epoch 682/1000
1052/1052 [==

Epoch 749/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9110 - val_loss: 11.3645
Epoch 750/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7052 - val_loss: 11.3074
Epoch 751/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0927 - val_loss: 11.3592
Epoch 752/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.4836 - val_loss: 11.4185
Epoch 753/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.8665 - val_loss: 10.9181
Epoch 754/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9833 - val_loss: 10.5688
Epoch 755/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.0309 - val_loss: 11.1466
Epoch 756/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.4727 - val_loss: 11.7891
Epoch 757/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.4166 - val_loss: 11.7515
Epoch 758/1000
1052/1052 [==

Epoch 900/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.8095 - val_loss: 10.5084
Epoch 901/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0477 - val_loss: 12.6898
Epoch 902/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0080 - val_loss: 10.4480
Epoch 903/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.9375 - val_loss: 10.3488
Epoch 904/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.7186 - val_loss: 10.6517
Epoch 905/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.4470 - val_loss: 11.4753
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2210 - val_loss: 10.0972
Epoch 907/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9275 - val_loss: 10.4636
Epoch 908/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7002 - val_loss: 10.3119
Epoch 909/1000
1052/1052 [==

Epoch 976/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8376 - val_loss: 11.6870
Epoch 977/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.4478 - val_loss: 11.5478
Epoch 978/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.9876 - val_loss: 9.9819
Epoch 979/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9660 - val_loss: 11.0166
Epoch 980/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7270 - val_loss: 10.0814
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.7277 - val_loss: 13.3384
Epoch 982/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8121 - val_loss: 11.3445
Epoch 983/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.8993 - val_loss: 13.0884
Epoch 984/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.2503 - val_loss: 10.2963
Epoch 985/1000
1052/1052 [===

[array([[ 1.0931299 , -1.585217  , -3.9492984 ,  3.9457076 ,  0.19601813],
        [ 0.12917024,  0.35905984,  0.10197406, -0.33409247,  0.24781229],
        [ 0.37755013,  0.28027803,  0.08867396,  0.55243546,  0.36083567],
        [-0.11255839,  0.0767654 ,  0.06075545, -0.14741232, -0.09606807],
        [ 1.0199596 , -0.16196637, -2.5203693 ,  0.45489472,  0.18659346]],
       dtype=float32),
 array([ 2.9997742 , -1.2472988 , -5.1978593 ,  4.852621  ,  0.20463541],
       dtype=float32),
 array([[ 0.9666616 , -1.0472397 , -0.6948292 , -1.3029972 , -1.4338678 ,
          0.9804574 , -1.5279957 ,  1.792749  , -1.4773501 ,  1.2949092 ,
         -1.4363718 , -1.6399841 ,  1.9651166 ,  1.5586802 ,  1.4115549 ],
        [ 0.7631275 , -0.84795517, -1.1737648 , -0.3553791 , -0.7445736 ,
          0.36059582, -0.9292283 ,  0.29607624, -1.1318733 ,  0.33572406,
         -0.48424637, -0.5592855 ,  0.17047863,  0.6634464 ,  0.6935452 ],
        [-1.0797013 ,  0.7789473 ,  0.32452092,  0.7525826

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure3.h5')

In [32]:
model_airfoil, history_airfoil=NN_model_structure_regression_4(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_8 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 225us/step - l

1052/1052 [==============================] - 0s 82us/step - loss: 14.3804 - val_loss: 18.0672
Epoch 68/1000
1052/1052 [==============================] - 0s 82us/step - loss: 14.1201 - val_loss: 17.3613
Epoch 69/1000
1052/1052 [==============================] - 0s 83us/step - loss: 14.1783 - val_loss: 16.7310
Epoch 70/1000
1052/1052 [==============================] - 0s 82us/step - loss: 14.0839 - val_loss: 17.7336
Epoch 71/1000
1052/1052 [==============================] - 0s 79us/step - loss: 13.8637 - val_loss: 17.3969
Epoch 72/1000
1052/1052 [==============================] - 0s 83us/step - loss: 14.0784 - val_loss: 16.5746
Epoch 73/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.6191 - val_loss: 17.1502
Epoch 74/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.5566 - val_loss: 16.8024
Epoch 75/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.6440 - val_loss: 18.0250
Epoch 76/1000
1052/1052 [=================

Epoch 143/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.0729 - val_loss: 12.4312
Epoch 144/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.1621 - val_loss: 10.5930
Epoch 145/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0799 - val_loss: 10.7473
Epoch 146/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3186 - val_loss: 11.1146
Epoch 147/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.3807 - val_loss: 11.0279
Epoch 148/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9779 - val_loss: 10.8111
Epoch 149/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1414 - val_loss: 11.9914
Epoch 150/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.8785 - val_loss: 11.5685
Epoch 151/1000
1052/1052 [==============================] - 0s 109us/step - loss: 8.9546 - val_loss: 10.8310
Epoch 152/1000
1052/1052 [=

Epoch 219/1000
1052/1052 [==============================] - 0s 103us/step - loss: 8.7184 - val_loss: 10.6647
Epoch 220/1000
1052/1052 [==============================] - 0s 112us/step - loss: 8.9236 - val_loss: 10.4144
Epoch 221/1000
1052/1052 [==============================] - 0s 108us/step - loss: 8.5249 - val_loss: 9.9768
Epoch 222/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.5238 - val_loss: 10.1948
Epoch 223/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.1126 - val_loss: 11.0944
Epoch 224/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.8688 - val_loss: 13.5153
Epoch 225/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.8995 - val_loss: 10.8430
Epoch 226/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.4065 - val_loss: 10.8108
Epoch 227/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.6734 - val_loss: 10.1903
Epoch 228/1000
1052/1052

Epoch 295/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.5310 - val_loss: 9.8326
Epoch 296/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.2539 - val_loss: 9.6591
Epoch 297/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.2931 - val_loss: 9.8446
Epoch 298/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3745 - val_loss: 9.5716
Epoch 299/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4157 - val_loss: 10.6942
Epoch 300/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.5186 - val_loss: 10.0832
Epoch 301/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6915 - val_loss: 9.7847
Epoch 302/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4709 - val_loss: 9.4126
Epoch 303/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3723 - val_loss: 9.5738
Epoch 304/1000
1052/1052 [=========

...

1052/1052 [==============================] - 0s 85us/step - loss: 7.7514 - val_loss: 9.6520
Epoch 524/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3968 - val_loss: 9.9563
Epoch 525/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.9991 - val_loss: 8.9780
Epoch 526/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.7554 - val_loss: 8.7223
Epoch 527/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1524 - val_loss: 9.9406
Epoch 528/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.4722 - val_loss: 9.9015
Epoch 529/1000
1052/1052 [==============================] - 0s 99us/step - loss: 7.6802 - val_loss: 9.4208
Epoch 530/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.3813 - val_loss: 9.7059
Epoch 531/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.9373 - val_loss: 8.9476
Epoch 532/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 7.7202 - val_loss: 8.5801
Epoch 676/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.9749 - val_loss: 8.7844
Epoch 677/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.9492 - val_loss: 10.7833
Epoch 678/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2882 - val_loss: 8.4685
Epoch 679/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8930 - val_loss: 9.6401
Epoch 680/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.7980 - val_loss: 8.9192
Epoch 681/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.8383 - val_loss: 9.2368
Epoch 682/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.8163 - val_loss: 11.2127
Epoch 683/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1790 - val_loss: 8.8122
Epoch 684/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 86us/step - loss: 7.6674 - val_loss: 9.1671
Epoch 828/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.7190 - val_loss: 9.0825
Epoch 829/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.8503 - val_loss: 9.0636
Epoch 830/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.5332 - val_loss: 9.1399
Epoch 831/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.6514 - val_loss: 10.0493
Epoch 832/1000
1052/1052 [==============================] - 0s 98us/step - loss: 7.7191 - val_loss: 9.1991
Epoch 833/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.9555 - val_loss: 10.2217
Epoch 834/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.9382 - val_loss: 8.9880
Epoch 835/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.7960 - val_loss: 8.8901
Epoch 836/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 85us/step - loss: 7.3215 - val_loss: 8.8990
Epoch 980/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.3472 - val_loss: 9.2289
Epoch 981/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.1532 - val_loss: 8.5461
Epoch 982/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.3344 - val_loss: 8.5550
Epoch 983/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.1760 - val_loss: 9.0634
Epoch 984/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.0448 - val_loss: 8.4419
Epoch 985/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.3134 - val_loss: 8.8617
Epoch 986/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.1824 - val_loss: 9.7055
Epoch 987/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.7728 - val_loss: 8.5778
Epoch 988/1000
1052/1052 [==========================

[array([[ 3.6629193e+00, -4.1821408e+00,  1.4003548e-01,  5.6738055e-01,
          4.7731609e+00,  1.0522735e+00,  1.9782355e-01,  2.3614011e+00,
         -2.7242017e-01,  5.6141192e-01],
        [ 1.3846671e-05, -3.6451843e-01,  2.6113147e-01,  5.8864814e-01,
         -2.3233098e-01,  1.7336996e-01,  2.8004992e-01, -8.6592466e-02,
         -7.0145893e-01,  1.2971787e+00],
        [ 9.3188393e-01,  8.5571177e-02,  4.3465877e-01,  7.6537108e-01,
          1.3316897e+00,  4.5016477e-01,  4.4844222e-01,  2.0404369e-02,
         -1.3066533e+00,  1.9121230e+00],
        [-2.1175379e-01, -4.2004141e-04, -1.1489381e+00, -9.5486480e-01,
         -1.7560390e-01, -3.1677046e-01, -1.0787916e+00, -1.1977392e-01,
          1.9373344e-01,  4.0366974e-01],
        [ 3.6936569e+00, -6.8076414e-01,  3.3399972e-01,  3.8299090e-01,
          1.9922523e-01,  5.9273535e-01,  3.3361945e-01,  2.2483425e-01,
         -8.8575995e-01,  6.5999871e-01]], dtype=float32),
 array([ 4.919988  , -4.6014085 , -3.986003

In [33]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure4.h5')

In [34]:
model_airfoil, history_airfoil=NN_model_structure_regression_5(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 240us/step - l

1052/1052 [==============================] - 0s 91us/step - loss: 13.9949 - val_loss: 16.9277
Epoch 68/1000
1052/1052 [==============================] - 0s 85us/step - loss: 13.7669 - val_loss: 16.7215
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.7395 - val_loss: 17.2666
Epoch 70/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.3479 - val_loss: 16.3929
Epoch 71/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.5503 - val_loss: 18.8789
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.2037 - val_loss: 16.5703
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.6478 - val_loss: 17.1923
Epoch 74/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.4984 - val_loss: 16.2088
Epoch 75/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.3331 - val_loss: 17.6439
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 92us/step - loss: 9.4965 - val_loss: 11.9099
Epoch 218/1000
1052/1052 [==============================] - 0s 97us/step - loss: 9.0958 - val_loss: 12.5061
Epoch 219/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.1619 - val_loss: 11.3471
Epoch 220/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.9929 - val_loss: 10.5916
Epoch 221/1000
1052/1052 [==============================] - 0s 102us/step - loss: 9.2964 - val_loss: 10.5999
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.0030 - val_loss: 10.8517
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.0695 - val_loss: 10.3595
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0601 - val_loss: 14.0655
Epoch 225/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.5330 - val_loss: 11.4674
Epoch 226/1000
1052/1052 [===============

1052/1052 [==============================] - 0s 87us/step - loss: 7.0496 - val_loss: 7.5110
Epoch 370/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.5038 - val_loss: 7.5899
Epoch 371/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.7578 - val_loss: 7.0897
Epoch 372/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.5727 - val_loss: 8.7963
Epoch 373/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.8030 - val_loss: 7.8483
Epoch 374/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4255 - val_loss: 9.2579
Epoch 375/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.7593 - val_loss: 7.4403
Epoch 376/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.3510 - val_loss: 8.8760
Epoch 377/1000
1052/1052 [==============================] - 0s 83us/step - loss: 6.9074 - val_loss: 8.8336
Epoch 378/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 103us/step - loss: 5.9817 - val_loss: 6.1282
Epoch 522/1000
1052/1052 [==============================] - 0s 102us/step - loss: 5.9538 - val_loss: 7.2494
Epoch 523/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.1012 - val_loss: 6.7864
Epoch 524/1000
1052/1052 [==============================] - 0s 106us/step - loss: 5.8613 - val_loss: 6.3980
Epoch 525/1000
1052/1052 [==============================] - 0s 101us/step - loss: 5.9027 - val_loss: 6.5703
Epoch 526/1000
1052/1052 [==============================] - 0s 114us/step - loss: 6.5907 - val_loss: 9.7176
Epoch 527/1000
1052/1052 [==============================] - 0s 110us/step - loss: 6.0362 - val_loss: 7.0352
Epoch 528/1000
1052/1052 [==============================] - 0s 110us/step - loss: 5.9950 - val_loss: 6.5171
Epoch 529/1000
1052/1052 [==============================] - 0s 99us/step - loss: 6.0777 - val_loss: 6.4024
Epoch 530/1000
1052/1052 [===================

1052/1052 [==============================] - 0s 87us/step - loss: 5.9410 - val_loss: 6.1929
Epoch 674/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.8229 - val_loss: 6.0715
Epoch 675/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.5587 - val_loss: 5.9616
Epoch 676/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6247 - val_loss: 6.0870
Epoch 677/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.6747 - val_loss: 7.9747
Epoch 678/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.6010 - val_loss: 5.9551
Epoch 679/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.0820 - val_loss: 7.0287
Epoch 680/1000
1052/1052 [==============================] - 0s 85us/step - loss: 6.0568 - val_loss: 6.4650
Epoch 681/1000
1052/1052 [==============================] - 0s 83us/step - loss: 6.6820 - val_loss: 7.1205
Epoch 682/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 5.7952 - val_loss: 6.3428
Epoch 826/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.5813 - val_loss: 5.9974
Epoch 827/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.4520 - val_loss: 5.8833
Epoch 828/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.2847 - val_loss: 5.2145
Epoch 829/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.6017 - val_loss: 5.9852
Epoch 830/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5959 - val_loss: 6.8105
Epoch 831/1000
1052/1052 [==============================] - 0s 82us/step - loss: 5.6951 - val_loss: 5.8493
Epoch 832/1000
1052/1052 [==============================] - 0s 81us/step - loss: 5.5401 - val_loss: 6.2510
Epoch 833/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.7025 - val_loss: 5.7217
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 89us/step - loss: 5.3796 - val_loss: 5.7502
Epoch 978/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.7186 - val_loss: 7.8630
Epoch 979/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.7949 - val_loss: 5.6389
Epoch 980/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.6462 - val_loss: 5.3292
Epoch 981/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.9208 - val_loss: 5.9430
Epoch 982/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.6447 - val_loss: 5.6682
Epoch 983/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5594 - val_loss: 8.0321
Epoch 984/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6945 - val_loss: 5.3313
Epoch 985/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3905 - val_loss: 5.7165
Epoch 986/1000
1052/1052 [==========================

[array([[ 4.3261294 , -3.7740855 ,  0.50674176,  0.25428903,  0.6039649 ,
          0.83112895,  1.094137  ,  0.29862598,  2.818076  ,  4.31269   ],
        [-0.13249727,  0.09763704,  0.9974108 ,  0.5220274 ,  0.4422099 ,
          0.4903943 ,  1.0849491 , -0.26379034, -0.1421228 ,  0.5473261 ],
        [-0.08075162, -1.5269315 ,  1.9977796 ,  1.669946  , -0.47504622,
          0.6857303 ,  0.39374834,  0.43975317,  0.06284396,  2.2318997 ],
        [-0.01838448,  0.15533598,  0.06709491, -0.14209753,  0.30370143,
         -1.1516814 , -0.41768652, -0.18161424, -0.07926843, -0.19530301],
        [ 2.199923  ,  0.13998659,  0.5756548 , -0.19102582,  0.93045783,
          1.1241857 ,  1.124133  ,  2.243338  ,  0.2018554 ,  0.5573553 ]],
       dtype=float32),
 array([ 5.1362696 , -5.7474537 , -5.304455  ,  1.1318691 , -5.1424193 ,
        -3.1003447 ,  0.34960106,  3.4770808 ,  1.6631205 ,  4.400375  ],
       dtype=float32),
 array([[-1.4864577 ,  1.9812807 ,  2.1445858 , -1.1233878 , 

In [35]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure5.h5')

In [36]:
model_airfoil, history_airfoil=NN_model_structure_regression_6(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000)
model_airfoil.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_12 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 253us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 12.3259 - val_loss: 14.4633
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.3567 - val_loss: 15.0813
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.1084 - val_loss: 14.5853
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.0842 - val_loss: 14.2448
Epoch 71/1000
1052/1052 [==============================] - 0s 87us/step - loss: 12.1357 - val_loss: 14.4968
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.7939 - val_loss: 15.6207
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.4061 - val_loss: 15.8255
Epoch 74/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.7720 - val_loss: 14.4477
Epoch 75/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.9288 - val_loss: 13.9776
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 100us/step - loss: 7.7888 - val_loss: 9.8952
Epoch 218/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.5362 - val_loss: 9.2846
Epoch 219/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.6397 - val_loss: 10.5439
Epoch 220/1000
1052/1052 [==============================] - 0s 100us/step - loss: 7.8926 - val_loss: 9.2809
Epoch 221/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.0022 - val_loss: 11.7729
Epoch 222/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.7029 - val_loss: 9.1327
Epoch 223/1000
1052/1052 [==============================] - 0s 97us/step - loss: 7.7970 - val_loss: 8.9732
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1609 - val_loss: 10.1266
Epoch 225/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.6332 - val_loss: 10.0987
Epoch 226/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 86us/step - loss: 7.8053 - val_loss: 9.3501
Epoch 370/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.2140 - val_loss: 7.7702
Epoch 371/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.5279 - val_loss: 8.1768
Epoch 372/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.5956 - val_loss: 8.9734
Epoch 373/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.5551 - val_loss: 8.0972
Epoch 374/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.0025 - val_loss: 9.4913
Epoch 375/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.1024 - val_loss: 8.0706
Epoch 376/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.1468 - val_loss: 7.8312
Epoch 377/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.1179 - val_loss: 8.1446
Epoch 378/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 84us/step - loss: 6.8783 - val_loss: 8.1269
Epoch 522/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.2680 - val_loss: 8.5662
Epoch 523/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.0316 - val_loss: 7.1819
Epoch 524/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9556 - val_loss: 6.7772
Epoch 525/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.2259 - val_loss: 7.0454
Epoch 526/1000
1052/1052 [==============================] - 0s 92us/step - loss: 6.0866 - val_loss: 8.0402
Epoch 527/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.2686 - val_loss: 6.7834
Epoch 528/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.3442 - val_loss: 6.9158
Epoch 529/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.7605 - val_loss: 7.3311
Epoch 530/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 6.4331 - val_loss: 7.5378
Epoch 674/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.0023 - val_loss: 5.8340
Epoch 675/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.2404 - val_loss: 7.7130
Epoch 676/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.8938 - val_loss: 6.7603
Epoch 677/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.0911 - val_loss: 5.8853
Epoch 678/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.8312 - val_loss: 6.1512
Epoch 679/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6740 - val_loss: 8.7376
Epoch 680/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.7648 - val_loss: 5.7841
Epoch 681/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.6924 - val_loss: 5.9831
Epoch 682/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 86us/step - loss: 5.5481 - val_loss: 5.6837
Epoch 826/1000
1052/1052 [==============================] - 0s 94us/step - loss: 5.5340 - val_loss: 6.3850
Epoch 827/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.4927 - val_loss: 6.0895
Epoch 828/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.8835 - val_loss: 9.6062
Epoch 829/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.1143 - val_loss: 6.5686
Epoch 830/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.7006 - val_loss: 5.8648
Epoch 831/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.4694 - val_loss: 6.5738
Epoch 832/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.8952 - val_loss: 5.8583
Epoch 833/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.8961 - val_loss: 6.5583
Epoch 834/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 5.8144 - val_loss: 6.0302
Epoch 978/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.9687 - val_loss: 7.9997
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.8932 - val_loss: 6.6664
Epoch 980/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.2242 - val_loss: 8.2443
Epoch 981/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.3670 - val_loss: 5.6643
Epoch 982/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.7972 - val_loss: 5.8406
Epoch 983/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.4023 - val_loss: 5.8882
Epoch 984/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.7214 - val_loss: 6.7398
Epoch 985/1000
1052/1052 [==============================] - 0s 97us/step - loss: 5.5048 - val_loss: 5.7326
Epoch 986/1000
1052/1052 [==========================

[array([[ 0.39857805, -2.874988  , -4.1522555 ,  3.5720425 ,  0.07070028,
         -0.48855358,  3.8396451 , -0.35278448, -1.1204606 , -0.24707508],
        [ 0.28992748,  0.16765077, -0.44597188, -0.07781048,  0.15109262,
         -1.036199  , -0.14750469,  0.08862649, -0.8946287 , -0.4639674 ],
        [ 0.412285  , -0.08402127, -2.061377  ,  1.6398654 ,  0.29448637,
         -1.8465567 , -0.08733255, -0.5006165 , -0.21092129, -1.5322034 ],
        [-0.5294359 ,  0.0754496 ,  0.2348902 , -0.15903318,  0.5030779 ,
         -0.7020627 , -0.05018321,  0.129734  ,  0.35723966,  0.11558765],
        [ 0.3802432 , -0.24111709, -0.7402314 , -0.20249675, -0.72839683,
         -0.6693197 ,  2.0102072 , -1.7434137 , -1.4586947 ,  0.02293078]],
       dtype=float32),
 array([-2.1692998 , -1.6766753 , -4.190005  ,  5.6368356 , -4.445437  ,
         5.6290693 ,  4.6944685 , -2.5627081 , -0.57562375, -1.1526766 ],
       dtype=float32),
 array([[ 0.4723931 , -0.66036904,  1.0017245 ,  0.12725852, 

In [37]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure6.h5')

In [38]:
model_autompg, history_autompg=NN_model_structure_regression_1(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_14 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 796us/step - loss: 5

Epoch 69/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5958 - val_loss: 9.6995
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6149 - val_loss: 9.8118
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4987 - val_loss: 9.7161
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8881 - val_loss: 10.0628
Epoch 73/1000
274/274 [==============================] - 0s 100us/step - loss: 6.7047 - val_loss: 9.6491
Epoch 74/1000
274/274 [==============================] - 0s 99us/step - loss: 6.4696 - val_loss: 9.6570
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4290 - val_loss: 9.4753
Epoch 76/1000
274/274 [==============================] - 0s 101us/step - loss: 6.4446 - val_loss: 9.5018
Epoch 77/1000
274/274 [==============================] - 0s 93us/step - loss: 6.4836 - val_loss: 9.5490
Epoch 78/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 95us/step - loss: 5.7364 - val_loss: 8.6195
Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7871 - val_loss: 8.9146
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7459 - val_loss: 8.7874
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6303 - val_loss: 8.4976
Epoch 228/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6305 - val_loss: 8.4623
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6418 - val_loss: 8.6307
Epoch 230/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6149 - val_loss: 9.0550
Epoch 231/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6603 - val_loss: 8.8974
Epoch 232/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7545 - val_loss: 8.4952
Epoch 233/1000
274/274 [==============================] - 0s 95us/step

274/274 [==============================] - 0s 95us/step - loss: 5.5555 - val_loss: 8.7638
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 5.6624 - val_loss: 8.8114
Epoch 304/1000
274/274 [==============================] - 0s 105us/step - loss: 5.6017 - val_loss: 8.5380
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7479 - val_loss: 8.7185
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9090 - val_loss: 9.1011
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7145 - val_loss: 8.7071
Epoch 308/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6476 - val_loss: 8.6085
Epoch 309/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6927 - val_loss: 8.3453
Epoch 310/1000
274/274 [==============================] - 0s 97us/step - loss: 5.8770 - val_loss: 9.0831
Epoch 311/1000
274/274 [==============================] - 0s 94us/s

274/274 [==============================] - 0s 95us/step - loss: 5.5023 - val_loss: 8.5733
Epoch 381/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7217 - val_loss: 8.8234
Epoch 382/1000
274/274 [==============================] - 0s 94us/step - loss: 5.6077 - val_loss: 8.4087
Epoch 383/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6625 - val_loss: 8.8301
Epoch 384/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4955 - val_loss: 8.5753
Epoch 385/1000
274/274 [==============================] - 0s 101us/step - loss: 5.5994 - val_loss: 8.9910
Epoch 386/1000
274/274 [==============================] - 0s 96us/step - loss: 5.5709 - val_loss: 8.3869
Epoch 387/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6439 - val_loss: 8.4960
Epoch 388/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4659 - val_loss: 8.9013
Epoch 389/1000
274/274 [==============================] - 0s 87us/ste

...

Epoch 536/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5514 - val_loss: 8.3512
Epoch 537/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4515 - val_loss: 8.4367
Epoch 538/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5273 - val_loss: 8.7352
Epoch 539/1000
274/274 [==============================] - 0s 93us/step - loss: 5.4667 - val_loss: 8.3989
Epoch 540/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5000 - val_loss: 8.4292
Epoch 541/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6565 - val_loss: 8.5284
Epoch 542/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4584 - val_loss: 8.5914
Epoch 543/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4685 - val_loss: 8.5561
Epoch 544/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4224 - val_loss: 8.3767
Epoch 545/1000
274/274 [==============================]

Epoch 614/1000
274/274 [==============================] - 0s 97us/step - loss: 5.4166 - val_loss: 8.9390
Epoch 615/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4406 - val_loss: 8.4694
Epoch 616/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8128 - val_loss: 8.4597
Epoch 617/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5661 - val_loss: 8.6703
Epoch 618/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8388 - val_loss: 8.7290
Epoch 619/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6726 - val_loss: 8.6414
Epoch 620/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7441 - val_loss: 8.3385
Epoch 621/1000
274/274 [==============================] - 0s 99us/step - loss: 5.8636 - val_loss: 8.8257
Epoch 622/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4520 - val_loss: 8.7298
Epoch 623/1000
274/274 [============================

Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4449 - val_loss: 8.9449
Epoch 770/1000
274/274 [==============================] - 0s 105us/step - loss: 5.4749 - val_loss: 8.4677
Epoch 771/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6071 - val_loss: 8.8617
Epoch 772/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6688 - val_loss: 8.2494
Epoch 773/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4280 - val_loss: 8.6613
Epoch 774/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6050 - val_loss: 8.5097
Epoch 775/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4608 - val_loss: 8.4443
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4120 - val_loss: 8.4007
Epoch 777/1000
274/274 [==============================] - 0s 113us/step - loss: 5.5085 - val_loss: 8.6314
Epoch 778/1000
274/274 [========================

Epoch 847/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3698 - val_loss: 8.5823
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5140 - val_loss: 8.4850
Epoch 849/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6418 - val_loss: 9.0535
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5820 - val_loss: 8.4573
Epoch 851/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3946 - val_loss: 8.3736
Epoch 852/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3985 - val_loss: 8.4595
Epoch 853/1000
274/274 [==============================] - 0s 99us/step - loss: 5.4027 - val_loss: 8.5185
Epoch 854/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7183 - val_loss: 8.5634
Epoch 855/1000
274/274 [==============================] - 0s 92us/step - loss: 5.3950 - val_loss: 8.4874
Epoch 856/1000
274/274 [==============================]

[array([[-1.59509325e+00,  1.32915854e-01,  1.50374532e-01,
         -1.60559261e+00,  1.66714025e+00],
        [-5.40645599e-01,  3.34446102e-01, -1.20253181e+00,
         -6.15950525e-01, -6.51584029e-01],
        [ 1.04397321e+00,  3.03861797e-01, -1.10061996e-01,
         -2.71698862e-01,  2.10035741e-01],
        [ 5.79888105e-01, -9.61458147e-01,  9.46186602e-01,
          6.88745439e-01, -1.10269701e+00],
        [ 1.58493564e-01,  3.13258708e-01,  2.62436122e-01,
          1.21346954e-03, -9.54081655e-01],
        [ 4.81030136e-01,  1.13828683e+00,  1.19745433e+00,
         -5.42439103e-01,  1.73349261e+00],
        [ 2.56063193e-01,  1.59793541e-01, -1.75328875e+00,
         -9.57204551e-02,  2.18215489e+00]], dtype=float32),
 array([-1.0935729 , -1.1934397 , -3.3114355 , -0.10305444,  3.3922274 ],
       dtype=float32),
 array([[-1.005919  ,  0.68058753, -1.0886734 ,  0.48754424, -0.06260633],
        [ 1.2751895 , -0.6617254 ,  0.7123754 , -1.6778141 ,  1.4914669 ],
        

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure1.h5')

In [40]:
model_autompg, history_autompg=NN_model_structure_regression_2(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 850us/step - loss:

274/274 [==============================] - 0s 95us/step - loss: 6.8493 - val_loss: 9.5397
Epoch 148/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3046 - val_loss: 9.1090
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0805 - val_loss: 9.4891
Epoch 150/1000
274/274 [==============================] - 0s 84us/step - loss: 6.0689 - val_loss: 9.2142
Epoch 151/1000
274/274 [==============================] - 0s 127us/step - loss: 6.0660 - val_loss: 9.2834
Epoch 152/1000
274/274 [==============================] - 0s 109us/step - loss: 6.1743 - val_loss: 9.3061
Epoch 153/1000
274/274 [==============================] - 0s 96us/step - loss: 6.0306 - val_loss: 9.5150
Epoch 154/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2312 - val_loss: 9.5048
Epoch 155/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2374 - val_loss: 9.4134
Epoch 156/1000
274/274 [==============================] - 0s 94us/st

274/274 [==============================] - 0s 99us/step - loss: 6.0172 - val_loss: 9.0738
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9596 - val_loss: 9.0944
Epoch 227/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0292 - val_loss: 9.0308
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9615 - val_loss: 9.2492
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9926 - val_loss: 9.2684
Epoch 230/1000
274/274 [==============================] - 0s 96us/step - loss: 6.0216 - val_loss: 9.2327
Epoch 231/1000
274/274 [==============================] - 0s 100us/step - loss: 5.9455 - val_loss: 9.3054
Epoch 232/1000
274/274 [==============================] - 0s 94us/step - loss: 5.8810 - val_loss: 9.2996
Epoch 233/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0568 - val_loss: 9.2581
Epoch 234/1000
274/274 [==============================] - 0s 91us/ste

Epoch 303/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7523 - val_loss: 8.8638
Epoch 304/1000
274/274 [==============================] - 0s 120us/step - loss: 5.7908 - val_loss: 8.8065
Epoch 305/1000
274/274 [==============================] - 0s 116us/step - loss: 5.8256 - val_loss: 8.5827
Epoch 306/1000
274/274 [==============================] - 0s 113us/step - loss: 5.8570 - val_loss: 8.8753
Epoch 307/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7759 - val_loss: 8.8519
Epoch 308/1000
274/274 [==============================] - 0s 109us/step - loss: 5.9049 - val_loss: 8.7853
Epoch 309/1000
274/274 [==============================] - 0s 109us/step - loss: 5.8899 - val_loss: 8.7182
Epoch 310/1000
274/274 [==============================] - 0s 117us/step - loss: 5.7115 - val_loss: 8.6641
Epoch 311/1000
274/274 [==============================] - 0s 113us/step - loss: 5.8424 - val_loss: 8.7593
Epoch 312/1000
274/274 [======================

274/274 [==============================] - 0s 98us/step - loss: 5.6409 - val_loss: 8.7392
Epoch 459/1000
274/274 [==============================] - 0s 93us/step - loss: 5.6203 - val_loss: 8.7447
Epoch 460/1000
274/274 [==============================] - 0s 96us/step - loss: 5.6316 - val_loss: 8.7620
Epoch 461/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6538 - val_loss: 8.7214
Epoch 462/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5657 - val_loss: 8.8331
Epoch 463/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5910 - val_loss: 8.8025
Epoch 464/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7433 - val_loss: 8.4121
Epoch 465/1000
274/274 [==============================] - 0s 97us/step - loss: 5.6952 - val_loss: 8.7810
Epoch 466/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5658 - val_loss: 8.7120
Epoch 467/1000
274/274 [==============================] - 0s 101us/ste

...

274/274 [==============================] - 0s 95us/step - loss: 5.6631 - val_loss: 9.2968
Epoch 615/1000
274/274 [==============================] - 0s 101us/step - loss: 5.9053 - val_loss: 9.1283
Epoch 616/1000
274/274 [==============================] - 0s 103us/step - loss: 5.8181 - val_loss: 9.2924
Epoch 617/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6327 - val_loss: 9.1898
Epoch 618/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6391 - val_loss: 9.1482
Epoch 619/1000
274/274 [==============================] - 0s 87us/step - loss: 5.7913 - val_loss: 9.8639
Epoch 620/1000
274/274 [==============================] - 0s 88us/step - loss: 5.6346 - val_loss: 9.3831
Epoch 621/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7981 - val_loss: 9.2743
Epoch 622/1000
274/274 [==============================] - 0s 93us/step - loss: 5.5542 - val_loss: 9.0564
Epoch 623/1000
274/274 [==============================] - 0s 98us/st

Epoch 692/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4545 - val_loss: 9.4770
Epoch 693/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4334 - val_loss: 9.5434
Epoch 694/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4172 - val_loss: 9.5604
Epoch 695/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3724 - val_loss: 9.5263
Epoch 696/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4717 - val_loss: 9.4003
Epoch 697/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3724 - val_loss: 9.4522
Epoch 698/1000
274/274 [==============================] - 0s 99us/step - loss: 5.4315 - val_loss: 9.2963
Epoch 699/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4708 - val_loss: 9.5606
Epoch 700/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4129 - val_loss: 9.6840
Epoch 701/1000
274/274 [============================

274/274 [==============================] - 0s 91us/step - loss: 5.4278 - val_loss: 10.6188
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2362 - val_loss: 10.4001
Epoch 848/1000
274/274 [==============================] - 0s 99us/step - loss: 5.1826 - val_loss: 10.2696
Epoch 849/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2283 - val_loss: 10.4458
Epoch 850/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1903 - val_loss: 10.4778
Epoch 851/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2311 - val_loss: 10.6569
Epoch 852/1000
274/274 [==============================] - 0s 96us/step - loss: 5.3397 - val_loss: 10.6355
Epoch 853/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1628 - val_loss: 10.5643
Epoch 854/1000
274/274 [==============================] - 0s 91us/step - loss: 5.1851 - val_loss: 10.4916
Epoch 855/1000
274/274 [==============================] - 0s

Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2904 - val_loss: 10.4841
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2261 - val_loss: 10.5726
Epoch 925/1000
274/274 [==============================] - 0s 97us/step - loss: 5.1530 - val_loss: 10.4436
Epoch 926/1000
274/274 [==============================] - 0s 93us/step - loss: 5.1552 - val_loss: 10.6376
Epoch 927/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1938 - val_loss: 10.8071
Epoch 928/1000
274/274 [==============================] - 0s 93us/step - loss: 5.2430 - val_loss: 10.6157
Epoch 929/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2027 - val_loss: 10.7877
Epoch 930/1000
274/274 [==============================] - 0s 94us/step - loss: 5.1061 - val_loss: 10.5996
Epoch 931/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1467 - val_loss: 10.7312
Epoch 932/1000
274/274 [=====================

[array([[ 1.8621572 , -0.94891536,  1.2792019 ,  2.536485  ,  0.971716  ],
        [-1.9282612 ,  0.44026133, -0.4402181 , -1.9349838 ,  0.5772517 ],
        [ 0.1986694 ,  0.35925952,  0.39911836,  1.9117271 , -0.8598615 ],
        [ 2.4493756 ,  0.36982045,  0.5048437 ,  2.0246198 , -1.7611738 ],
        [-1.4726099 ,  0.6136756 ,  0.5517549 ,  0.02647304, -0.01828146],
        [-1.0734944 , -0.33369312, -1.0108453 , -1.0291033 ,  1.3529366 ],
        [ 0.5445342 , -1.2267237 , -1.2415302 , -0.7626701 ,  0.77862585]],
       dtype=float32),
 array([-1.9989395, -1.5158943,  2.7006118,  2.1345346, -2.4006443],
       dtype=float32),
 array([[-0.26419547, -0.77168554,  0.17058145,  0.1614366 ,  0.7320283 ,
         -0.21614407,  0.21548632, -0.5747494 ,  0.69486046, -0.07920573],
        [ 0.23817292, -0.4458577 , -0.5880809 ,  0.29146042,  0.80989975,
         -0.30275044, -0.14817785, -0.71266186, -0.18011235, -0.08997463],
        [-0.57301396,  0.6942921 ,  0.09045097, -0.05394081, 

In [41]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure2.h5')

In [42]:
model_autompg, history_autompg=NN_model_structure_regression_3(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_18 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 16        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 909us/step - loss:

274/274 [==============================] - 0s 95us/step - loss: 6.0602 - val_loss: 9.9545
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4979 - val_loss: 9.8995
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1038 - val_loss: 10.0084
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0750 - val_loss: 9.9937
Epoch 151/1000
274/274 [==============================] - 0s 94us/step - loss: 6.0216 - val_loss: 10.0494
Epoch 152/1000
274/274 [==============================] - 0s 97us/step - loss: 6.0228 - val_loss: 10.0152
Epoch 153/1000
274/274 [==============================] - 0s 96us/step - loss: 5.9480 - val_loss: 9.9850
Epoch 154/1000
274/274 [==============================] - 0s 100us/step - loss: 6.1055 - val_loss: 9.9262
Epoch 155/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1366 - val_loss: 9.8096
Epoch 156/1000
274/274 [==============================] - 0s 95u

Epoch 302/1000
274/274 [==============================] - 0s 109us/step - loss: 5.5369 - val_loss: 8.9405
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5177 - val_loss: 9.0040
Epoch 304/1000
274/274 [==============================] - 0s 109us/step - loss: 5.5439 - val_loss: 8.9542
Epoch 305/1000
274/274 [==============================] - 0s 113us/step - loss: 5.4599 - val_loss: 9.0097
Epoch 306/1000
274/274 [==============================] - 0s 113us/step - loss: 5.6905 - val_loss: 8.7818
Epoch 307/1000
274/274 [==============================] - 0s 110us/step - loss: 5.8953 - val_loss: 9.1509
Epoch 308/1000
274/274 [==============================] - 0s 116us/step - loss: 5.5058 - val_loss: 8.9672
Epoch 309/1000
274/274 [==============================] - 0s 107us/step - loss: 5.4948 - val_loss: 9.1006
Epoch 310/1000
274/274 [==============================] - 0s 113us/step - loss: 5.4853 - val_loss: 9.0322
Epoch 311/1000
274/274 [======================

Epoch 457/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2745 - val_loss: 8.5973
Epoch 458/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2855 - val_loss: 8.9812
Epoch 459/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3487 - val_loss: 8.9717
Epoch 460/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2779 - val_loss: 8.7128
Epoch 461/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3719 - val_loss: 8.9644
Epoch 462/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3723 - val_loss: 8.6785
Epoch 463/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2961 - val_loss: 8.7527
Epoch 464/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2391 - val_loss: 8.7150
Epoch 465/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4546 - val_loss: 9.0428
Epoch 466/1000
274/274 [==============================]

Epoch 612/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4039 - val_loss: 8.7485
Epoch 613/1000
274/274 [==============================] - 0s 94us/step - loss: 5.2465 - val_loss: 9.5812
Epoch 614/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2483 - val_loss: 8.8515
Epoch 615/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4136 - val_loss: 9.7163
Epoch 616/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2703 - val_loss: 8.7667
Epoch 617/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3288 - val_loss: 9.4186
Epoch 618/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6583 - val_loss: 9.0337
Epoch 619/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8182 - val_loss: 10.6559
Epoch 620/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6812 - val_loss: 8.9649
Epoch 621/1000
274/274 [=============================

Epoch 690/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2417 - val_loss: 9.0401
Epoch 691/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1948 - val_loss: 9.1176
Epoch 692/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2333 - val_loss: 9.2576
Epoch 693/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2113 - val_loss: 9.1851
Epoch 694/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2634 - val_loss: 9.0004
Epoch 695/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3077 - val_loss: 9.0478
Epoch 696/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3297 - val_loss: 9.2954
Epoch 697/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2837 - val_loss: 9.1188
Epoch 698/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2729 - val_loss: 9.0219
Epoch 699/1000
274/274 [==============================

Epoch 768/1000
274/274 [==============================] - 0s 99us/step - loss: 5.8386 - val_loss: 8.8106
Epoch 769/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4919 - val_loss: 9.6655
Epoch 770/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2291 - val_loss: 9.0810
Epoch 771/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1748 - val_loss: 9.4199
Epoch 772/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2102 - val_loss: 9.1575
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2575 - val_loss: 9.0017
Epoch 774/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2741 - val_loss: 9.0950
Epoch 775/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2275 - val_loss: 9.2051
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2298 - val_loss: 9.1894
Epoch 777/1000
274/274 [==============================

274/274 [==============================] - 0s 100us/step - loss: 5.1584 - val_loss: 9.4885
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1566 - val_loss: 9.1284
Epoch 925/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2404 - val_loss: 9.1530
Epoch 926/1000
274/274 [==============================] - 0s 110us/step - loss: 5.1914 - val_loss: 9.7069
Epoch 927/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5140 - val_loss: 8.9464
Epoch 928/1000
274/274 [==============================] - 0s 97us/step - loss: 5.3342 - val_loss: 9.4436
Epoch 929/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5126 - val_loss: 9.1659
Epoch 930/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3770 - val_loss: 9.5531
Epoch 931/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3115 - val_loss: 9.0736
Epoch 932/1000
274/274 [==============================] - 0s 98us/st

[array([[ 1.0505302 ,  0.3533768 , -0.13521643,  2.7658608 ,  0.97286737],
        [-2.109175  ,  1.3434516 , -0.54942197,  1.9954491 , -0.46865967],
        [ 0.18010966,  0.971631  ,  1.0157697 ,  1.9750657 , -0.07710479],
        [ 3.138973  , -1.8404623 ,  1.14082   , -2.7908416 , -0.22023597],
        [-1.5652248 ,  0.5182006 , -0.26527432,  0.8696655 , -0.58190536],
        [-1.6834608 , -0.80253536, -1.4577963 ,  1.5871503 ,  0.6569788 ],
        [-0.21412319, -1.043322  , -0.4362392 , -1.4875423 ,  1.2580156 ]],
       dtype=float32),
 array([-1.4607203,  0.9350687,  2.5024896, -0.5237697,  1.5457788],
       dtype=float32),
 array([[-0.09803031, -0.7969602 , -0.10738765,  0.11986196,  0.01278282,
         -0.35776636, -0.6091727 ,  0.03267024, -0.1084067 , -0.13089083,
         -0.76149297, -0.31194782, -0.20325273, -0.18411921,  0.61645716],
        [ 0.27926522,  0.83242124,  0.7231125 , -0.32267728, -0.24925843,
         -0.01952912,  0.7441183 ,  0.42166013,  0.06233505,  

In [43]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure3.h5')

In [44]:
model_autompg, history_autompg=NN_model_structure_regression_4(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_20 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 995us/step - loss

274/274 [==============================] - 0s 106us/step - loss: 7.0590 - val_loss: 8.8177
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9738 - val_loss: 8.7863
Epoch 71/1000
274/274 [==============================] - 0s 101us/step - loss: 7.0380 - val_loss: 8.6611
Epoch 72/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1006 - val_loss: 8.5166
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.0124 - val_loss: 8.5545
Epoch 74/1000
274/274 [==============================] - 0s 124us/step - loss: 6.9524 - val_loss: 8.6224
Epoch 75/1000
274/274 [==============================] - 0s 120us/step - loss: 7.3334 - val_loss: 8.5522
Epoch 76/1000
274/274 [==============================] - 0s 127us/step - loss: 7.0460 - val_loss: 8.6274
Epoch 77/1000
274/274 [==============================] - 0s 102us/step - loss: 7.1370 - val_loss: 8.5851
Epoch 78/1000
274/274 [==============================] - 0s 98us/step - 

Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6028 - val_loss: 7.5012
Epoch 226/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5942 - val_loss: 7.2672
Epoch 227/1000
274/274 [==============================] - 0s 104us/step - loss: 5.6486 - val_loss: 7.4468
Epoch 228/1000
274/274 [==============================] - 0s 96us/step - loss: 5.5175 - val_loss: 7.6055
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6610 - val_loss: 7.2470
Epoch 230/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8357 - val_loss: 7.3344
Epoch 231/1000
274/274 [==============================] - 0s 94us/step - loss: 5.6005 - val_loss: 7.1716
Epoch 232/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7458 - val_loss: 7.4156
Epoch 233/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4990 - val_loss: 7.0242
Epoch 234/1000
274/274 [============================

Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1170 - val_loss: 7.4737
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0417 - val_loss: 7.6562
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0552 - val_loss: 7.4647
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2225 - val_loss: 7.5155
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4088 - val_loss: 7.3332
Epoch 308/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0350 - val_loss: 7.5245
Epoch 309/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9943 - val_loss: 7.6190
Epoch 310/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1414 - val_loss: 8.0457
Epoch 311/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1932 - val_loss: 7.4715
Epoch 312/1000
274/274 [============================

274/274 [==============================] - 0s 102us/step - loss: 4.1938 - val_loss: 7.9575
Epoch 458/1000
274/274 [==============================] - 0s 105us/step - loss: 4.2085 - val_loss: 7.6822
Epoch 459/1000
274/274 [==============================] - 0s 100us/step - loss: 4.1684 - val_loss: 7.8183
Epoch 460/1000
274/274 [==============================] - 0s 105us/step - loss: 4.2550 - val_loss: 8.0975
Epoch 461/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3791 - val_loss: 7.7860
Epoch 462/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2644 - val_loss: 8.0699
Epoch 463/1000
274/274 [==============================] - 0s 105us/step - loss: 4.3239 - val_loss: 8.0446
Epoch 464/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2281 - val_loss: 8.0947
Epoch 465/1000
274/274 [==============================] - 0s 106us/step - loss: 4.4994 - val_loss: 7.6966
Epoch 466/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 102us/step - loss: 4.0036 - val_loss: 8.3826
Epoch 612/1000
274/274 [==============================] - 0s 101us/step - loss: 4.0304 - val_loss: 7.7828
Epoch 613/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0549 - val_loss: 8.8563
Epoch 614/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3853 - val_loss: 8.1036
Epoch 615/1000
274/274 [==============================] - 0s 97us/step - loss: 4.0193 - val_loss: 8.4506
Epoch 616/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9286 - val_loss: 7.9952
Epoch 617/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9123 - val_loss: 8.0809
Epoch 618/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8426 - val_loss: 7.8714
Epoch 619/1000
274/274 [==============================] - 0s 99us/step - loss: 3.8638 - val_loss: 8.0559
Epoch 620/1000
274/274 [==============================] - 0s 98u

274/274 [==============================] - 0s 106us/step - loss: 3.7950 - val_loss: 9.3505
Epoch 766/1000
274/274 [==============================] - 0s 146us/step - loss: 3.7507 - val_loss: 9.0154
Epoch 767/1000
274/274 [==============================] - 0s 106us/step - loss: 3.7444 - val_loss: 9.5358
Epoch 768/1000
274/274 [==============================] - 0s 116us/step - loss: 4.0216 - val_loss: 8.9744
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9131 - val_loss: 8.9035
Epoch 770/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7422 - val_loss: 8.7820
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7059 - val_loss: 8.7413
Epoch 772/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7213 - val_loss: 8.8025
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7332 - val_loss: 8.8856
Epoch 774/1000
274/274 [==============================] - 0s 102

274/274 [==============================] - 0s 113us/step - loss: 3.8652 - val_loss: 9.2371
Epoch 920/1000
274/274 [==============================] - 0s 113us/step - loss: 3.6782 - val_loss: 9.5324
Epoch 921/1000
274/274 [==============================] - 0s 109us/step - loss: 3.6824 - val_loss: 9.4482
Epoch 922/1000
274/274 [==============================] - 0s 105us/step - loss: 3.7099 - val_loss: 9.7418
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8316 - val_loss: 9.3837
Epoch 924/1000
274/274 [==============================] - 0s 102us/step - loss: 3.6565 - val_loss: 9.3757
Epoch 925/1000
274/274 [==============================] - 0s 102us/step - loss: 3.5885 - val_loss: 9.4463
Epoch 926/1000
274/274 [==============================] - 0s 116us/step - loss: 3.6480 - val_loss: 9.3341
Epoch 927/1000
274/274 [==============================] - 0s 135us/step - loss: 3.6823 - val_loss: 9.3021
Epoch 928/1000
274/274 [==============================] - 0s 1

[array([[-1.8734475 , -6.421421  ,  1.161752  ,  2.4628744 ,  2.7281158 ,
         -1.9099524 , -2.5594108 ,  0.44500044,  1.217952  ,  1.2142584 ],
        [ 1.56422   ,  4.8801527 ,  0.98925483, -0.5478158 ,  0.21167165,
          0.02454631, -1.5325325 , -0.60694724,  1.3101336 , -1.7040946 ],
        [-0.69268596, -1.6563165 , -1.6078761 , -0.7843384 , -0.19719788,
          2.7005916 ,  1.891307  ,  0.92202884,  1.8089575 ,  0.578264  ],
        [ 0.8683333 , -2.7352185 , -0.48507917,  1.5071272 , -1.4074535 ,
         -1.2575696 , -0.4500129 , -0.21918994,  0.09280241,  1.5180876 ],
        [ 0.26187485, -0.46950328,  0.88803357, -0.529626  ,  1.1293739 ,
         -0.64260375,  0.22745948,  0.8429583 ,  0.74788487, -0.75852484],
        [-0.32987183,  0.989335  , -1.0137888 , -1.623535  , -1.3660762 ,
          3.1294386 , -1.2221633 , -0.29628396,  1.0580381 ,  0.59780824],
        [ 0.63854307,  1.3310467 ,  0.962137  ,  0.80481625, -1.597244  ,
          3.5222404 , -0.1600126

In [45]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure4.h5')

In [46]:
model_autompg, history_autompg=NN_model_structure_regression_5(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

Epoch 69/1000
274/274 [==============================] - 0s 97us/step - loss: 6.0744 - val_loss: 8.5025
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3839 - val_loss: 8.8189
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6111 - val_loss: 8.9096
Epoch 72/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6948 - val_loss: 8.5152
Epoch 73/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0562 - val_loss: 8.4321
Epoch 74/1000
274/274 [==============================] - 0s 113us/step - loss: 6.0234 - val_loss: 8.2808
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9929 - val_loss: 8.4918
Epoch 76/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2376 - val_loss: 8.8814
Epoch 77/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3296 - val_loss: 8.5071
Epoch 78/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 95us/step - loss: 5.2858 - val_loss: 8.2997
Epoch 224/1000
274/274 [==============================] - 0s 115us/step - loss: 5.2582 - val_loss: 8.1034
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3997 - val_loss: 8.1096
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2980 - val_loss: 8.2123
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4284 - val_loss: 8.1367
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2567 - val_loss: 8.1453
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3472 - val_loss: 8.0794
Epoch 230/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2085 - val_loss: 8.1586
Epoch 231/1000
274/274 [==============================] - 0s 105us/step - loss: 5.2700 - val_loss: 7.9606
Epoch 232/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 109us/step - loss: 4.9208 - val_loss: 8.5961
Epoch 378/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7999 - val_loss: 8.5287
Epoch 379/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9856 - val_loss: 8.6835
Epoch 380/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9718 - val_loss: 8.5471
Epoch 381/1000
274/274 [==============================] - 0s 109us/step - loss: 4.7970 - val_loss: 8.4566
Epoch 382/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8562 - val_loss: 8.1388
Epoch 383/1000
274/274 [==============================] - 0s 98us/step - loss: 4.8530 - val_loss: 8.5416
Epoch 384/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1592 - val_loss: 8.9409
Epoch 385/1000
274/274 [==============================] - 0s 107us/step - loss: 4.9279 - val_loss: 8.6379
Epoch 386/1000
274/274 [==============================] - 0s 98

274/274 [==============================] - 0s 106us/step - loss: 4.5727 - val_loss: 8.8775
Epoch 532/1000
274/274 [==============================] - 0s 106us/step - loss: 4.6018 - val_loss: 9.1001
Epoch 533/1000
274/274 [==============================] - 0s 109us/step - loss: 4.5837 - val_loss: 9.1844
Epoch 534/1000
274/274 [==============================] - 0s 106us/step - loss: 4.5463 - val_loss: 9.0217
Epoch 535/1000
274/274 [==============================] - 0s 103us/step - loss: 4.7627 - val_loss: 9.0490
Epoch 536/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6522 - val_loss: 9.1114
Epoch 537/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5244 - val_loss: 9.3211
Epoch 538/1000
274/274 [==============================] - 0s 106us/step - loss: 4.6063 - val_loss: 9.2372
Epoch 539/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7855 - val_loss: 9.1567
Epoch 540/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 102us/step - loss: 4.3488 - val_loss: 9.2215
Epoch 686/1000
274/274 [==============================] - 0s 106us/step - loss: 4.3486 - val_loss: 8.9846
Epoch 687/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4385 - val_loss: 9.0553
Epoch 688/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4061 - val_loss: 9.0579
Epoch 689/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5062 - val_loss: 9.6327
Epoch 690/1000
274/274 [==============================] - 0s 106us/step - loss: 4.6085 - val_loss: 9.4062
Epoch 691/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5312 - val_loss: 9.1395
Epoch 692/1000
274/274 [==============================] - 0s 106us/step - loss: 4.4196 - val_loss: 9.0061
Epoch 693/1000
274/274 [==============================] - 0s 105us/step - loss: 4.3132 - val_loss: 9.1523
Epoch 694/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 116us/step - loss: 4.2153 - val_loss: 9.0466
Epoch 840/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2366 - val_loss: 8.9588
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.6293 - val_loss: 9.9120
Epoch 842/1000
274/274 [==============================] - 0s 109us/step - loss: 4.9465 - val_loss: 9.3457
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3303 - val_loss: 9.8750
Epoch 844/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1508 - val_loss: 9.2852
Epoch 845/1000
274/274 [==============================] - 0s 106us/step - loss: 4.4677 - val_loss: 9.3099
Epoch 846/1000
274/274 [==============================] - 0s 110us/step - loss: 4.2034 - val_loss: 9.1575
Epoch 847/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2736 - val_loss: 9.1798
Epoch 848/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 102us/step - loss: 4.0652 - val_loss: 9.5598
Epoch 994/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1602 - val_loss: 10.3415
Epoch 995/1000
274/274 [==============================] - 0s 100us/step - loss: 4.0335 - val_loss: 9.4829
Epoch 996/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2142 - val_loss: 9.9590
Epoch 997/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1586 - val_loss: 9.4444
Epoch 998/1000
274/274 [==============================] - 0s 105us/step - loss: 4.0958 - val_loss: 9.7333
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0566 - val_loss: 9.8629
Epoch 1000/1000
274/274 [==============================] - 0s 104us/step - loss: 4.0079 - val_loss: 9.5620
7.129009448875816


[array([[ 4.3556862e+00, -9.8558676e-01,  3.4957063e+00, -2.5149055e+00,
          4.1072149e+00, -1.4654508e+00, -3.6878756e-01,  7.3338300e-01,
         -2.8168871e+00,  9.4766253e-01],
        [-4.0873653e-01,  9.5570225e-01, -2.5147905e+00,  2.1838560e+00,
         -2.3769341e+00,  5.5652970e-01, -1.1814021e+00,  1.1163929e+00,
          1.0337913e+00, -1.4861494e+00],
        [-1.5664078e+00,  5.7811904e-01,  1.5507472e+00,  1.4246424e+00,
          4.7220376e-01, -1.0173656e+00, -5.6427622e-01, -1.7438691e+00,
          2.3114300e+00,  1.0944103e+00],
        [ 4.3667011e-02, -7.3571140e-01,  1.1948775e+00, -2.3046863e+00,
          2.0765352e+00,  7.9192877e-01,  1.1755912e+00, -9.5961010e-01,
         -8.2030541e-01, -2.5274384e-01],
        [-4.4600233e-01,  5.8837116e-01,  3.5861680e-01,  1.2449651e+00,
         -2.8521671e+00, -6.6574830e-01,  1.7491629e+00,  1.9707493e-01,
          7.3478633e-01,  3.1479467e-03],
        [ 1.5449178e+00, -1.2813964e-01, -5.2312595e-01, -1.

In [47]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure5.h5')

In [48]:
model_autompg, history_autompg=NN_model_structure_regression_6(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000)
model_autompg.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_23 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_24 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 16        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 106us/step - loss: 5.5644 - val_loss: 8.9928
Epoch 146/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4713 - val_loss: 8.7769
Epoch 147/1000
274/274 [==============================] - 0s 105us/step - loss: 5.6286 - val_loss: 10.3621
Epoch 148/1000
274/274 [==============================] - 0s 108us/step - loss: 5.6572 - val_loss: 9.1683
Epoch 149/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5449 - val_loss: 8.9705
Epoch 150/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4875 - val_loss: 9.6443
Epoch 151/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6101 - val_loss: 8.8866
Epoch 152/1000
274/274 [==============================] - 0s 109us/step - loss: 5.8253 - val_loss: 10.2936
Epoch 153/1000
274/274 [==============================] - 0s 103us/step - loss: 5.5676 - val_loss: 8.6869
Epoch 154/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 105us/step - loss: 4.6818 - val_loss: 9.3267
Epoch 300/1000
274/274 [==============================] - 0s 106us/step - loss: 4.7148 - val_loss: 9.5371
Epoch 301/1000
274/274 [==============================] - 0s 109us/step - loss: 4.7661 - val_loss: 9.2620
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6816 - val_loss: 9.3831
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 4.6446 - val_loss: 8.9768
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6922 - val_loss: 9.5400
Epoch 305/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9837 - val_loss: 9.9628
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9312 - val_loss: 9.2931
Epoch 307/1000
274/274 [==============================] - 0s 106us/step - loss: 4.8056 - val_loss: 9.8737
Epoch 308/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 104us/step - loss: 4.3343 - val_loss: 10.0348
Epoch 454/1000
274/274 [==============================] - 0s 114us/step - loss: 4.5227 - val_loss: 9.2831
Epoch 455/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2939 - val_loss: 9.5876
Epoch 456/1000
274/274 [==============================] - 0s 105us/step - loss: 4.3295 - val_loss: 9.8713
Epoch 457/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1992 - val_loss: 9.6501
Epoch 458/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1859 - val_loss: 9.3151
Epoch 459/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2080 - val_loss: 9.6513
Epoch 460/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2012 - val_loss: 9.3797
Epoch 461/1000
274/274 [==============================] - 0s 105us/step - loss: 4.5742 - val_loss: 11.4702
Epoch 462/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 108us/step - loss: 4.5252 - val_loss: 11.7044
Epoch 531/1000
274/274 [==============================] - 0s 109us/step - loss: 4.3284 - val_loss: 10.0544
Epoch 532/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5346 - val_loss: 9.7475
Epoch 533/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0208 - val_loss: 11.2219
Epoch 534/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2655 - val_loss: 9.8997
Epoch 535/1000
274/274 [==============================] - 0s 109us/step - loss: 4.4190 - val_loss: 10.8918
Epoch 536/1000
274/274 [==============================] - 0s 106us/step - loss: 4.2155 - val_loss: 10.6048
Epoch 537/1000
274/274 [==============================] - 0s 116us/step - loss: 4.2274 - val_loss: 10.2864
Epoch 538/1000
274/274 [==============================] - 0s 105us/step - loss: 4.1338 - val_loss: 10.4832
Epoch 539/1000
274/274 [==============================

Epoch 607/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9293 - val_loss: 10.8746
Epoch 608/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0530 - val_loss: 11.5257
Epoch 609/1000
274/274 [==============================] - 0s 102us/step - loss: 4.3897 - val_loss: 11.9081
Epoch 610/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2310 - val_loss: 11.4008
Epoch 611/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0820 - val_loss: 10.9405
Epoch 612/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2615 - val_loss: 10.8357
Epoch 613/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1737 - val_loss: 11.6387
Epoch 614/1000
274/274 [==============================] - 0s 135us/step - loss: 4.0288 - val_loss: 11.0453
Epoch 615/1000
274/274 [==============================] - 0s 118us/step - loss: 4.1331 - val_loss: 10.8045
Epoch 616/1000
274/274 [==============

274/274 [==============================] - 0s 101us/step - loss: 3.8415 - val_loss: 12.1648
Epoch 760/1000
274/274 [==============================] - 0s 109us/step - loss: 3.8953 - val_loss: 12.1328
Epoch 761/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9024 - val_loss: 12.1086
Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8724 - val_loss: 12.2294
Epoch 763/1000
274/274 [==============================] - 0s 109us/step - loss: 3.7927 - val_loss: 11.9306
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 3.8271 - val_loss: 12.1692
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8829 - val_loss: 11.9100
Epoch 766/1000
274/274 [==============================] - 0s 100us/step - loss: 3.9149 - val_loss: 12.0983
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2165 - val_loss: 11.6238
Epoch 768/1000
274/274 [=============================

Epoch 836/1000
274/274 [==============================] - 0s 113us/step - loss: 4.1553 - val_loss: 13.2124
Epoch 837/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9833 - val_loss: 11.7902
Epoch 838/1000
274/274 [==============================] - 0s 106us/step - loss: 3.7298 - val_loss: 12.2273
Epoch 839/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9555 - val_loss: 11.7172
Epoch 840/1000
274/274 [==============================] - 0s 113us/step - loss: 3.9850 - val_loss: 11.7956
Epoch 841/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9410 - val_loss: 12.1545
Epoch 842/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9980 - val_loss: 12.6266
Epoch 843/1000
274/274 [==============================] - 0s 109us/step - loss: 4.0537 - val_loss: 12.0363
Epoch 844/1000
274/274 [==============================] - 0s 109us/step - loss: 3.9417 - val_loss: 12.2144
Epoch 845/1000
274/274 [=============

Epoch 989/1000
274/274 [==============================] - 0s 120us/step - loss: 3.7390 - val_loss: 11.4907
Epoch 990/1000
274/274 [==============================] - 0s 116us/step - loss: 3.9799 - val_loss: 11.8065
Epoch 991/1000
274/274 [==============================] - 0s 124us/step - loss: 3.8006 - val_loss: 11.7417
Epoch 992/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1923 - val_loss: 11.5913
Epoch 993/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0162 - val_loss: 11.5335
Epoch 994/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8471 - val_loss: 11.5371
Epoch 995/1000
274/274 [==============================] - 0s 115us/step - loss: 3.8924 - val_loss: 11.5389
Epoch 996/1000
274/274 [==============================] - 0s 113us/step - loss: 3.7849 - val_loss: 12.2400
Epoch 997/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8439 - val_loss: 11.1868
Epoch 998/1000
274/274 [==============

[array([[ 0.05218785, -4.439589  ,  1.184245  , -0.32698205, -1.9974234 ,
         -1.882257  , -0.8196187 , -0.21171416, -1.1699376 ,  0.5623695 ],
        [-0.8764483 ,  3.2636888 , -1.4104617 ,  0.20296542, -1.9889956 ,
         -0.8744576 ,  1.8381467 , -0.40577483,  0.8736507 , -2.3100579 ],
        [-0.27670828, -0.42422557, -0.55878794, -0.52224237, -2.5005019 ,
          1.1224926 ,  0.8319177 ,  0.20461716,  0.07792918,  1.2341498 ],
        [ 1.5049043 , -2.7252157 ,  1.696431  , -1.0925831 ,  0.7352186 ,
          1.5018852 , -0.48673   ,  0.16460963, -0.936903  ,  3.003468  ],
        [-1.3972381 ,  0.52844405, -0.32371458, -1.2367066 , -0.03497618,
          0.19337356, -0.6444655 ,  0.19330864,  1.0046711 , -0.80353117],
        [ 0.8101482 ,  1.0985392 , -2.1631794 ,  1.1020138 ,  1.3484505 ,
          1.2724284 ,  0.46078345, -0.02956701, -0.3547916 , -1.2015022 ],
        [ 1.640306  ,  1.1607018 ,  0.03521889, -0.84423286, -0.720385  ,
         -0.68723327, -0.0912625

In [49]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure6.h5')

In [50]:
model_automobile, history_automobile=NN_model_structure_regression_1(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 105us/step - loss: 0.0063 - val_loss: 0.0049
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0052
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0046
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0047
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0049
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0049
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0047
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0049
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0063 - val_loss: 0.0046
Epoch 78/1000
143/143 [==============================] - 0s 118us/ste

143/143 [==============================] - 0s 116us/step - loss: 0.0049 - val_loss: 0.0040
Epoch 224/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0047 - val_loss: 0.0044
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0044
Epoch 226/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 227/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0041
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0039
Epoch 229/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0045 - val_loss: 0.0041
Epoch 230/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0044 - val_loss: 0.0038
Epoch 231/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0044 - val_loss: 0.0040
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0040
Epoch 378/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0037 - val_loss: 0.0038
Epoch 379/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0035
Epoch 380/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0043
Epoch 381/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 382/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0036 - val_loss: 0.0042
Epoch 383/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0036 - val_loss: 0.0033
Epoch 384/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0035
Epoch 385/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0038 - val_loss: 0.0045
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0033
Epoch 532/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0032 - val_loss: 0.0038
Epoch 533/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0039
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0038
Epoch 535/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0035
Epoch 536/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 537/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0031 - val_loss: 0.0037
Epoch 538/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 539/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0033
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0036
Epoch 686/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0043
Epoch 687/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 688/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0043
Epoch 689/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 690/1000
143/143 [==============================] - 0s 181us/step - loss: 0.0028 - val_loss: 0.0035
Epoch 691/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0031 - val_loss: 0.0042
Epoch 692/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0037
Epoch 693/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0029 - val_loss: 0.0036
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0025 - val_loss: 0.0046
Epoch 840/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0027 - val_loss: 0.0037
Epoch 841/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0030 - val_loss: 0.0035
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0039
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0045
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0037
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0057
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0036
Epoch 847/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0039
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0066
Epoch 995/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0025 - val_loss: 0.0041
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0039
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0051
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0041
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 1000/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0057
0.006008231546729803


[array([[ 1.0967789 , -0.85125566, -0.3005834 , -0.14808619, -0.260356  ],
        [ 0.58913124, -0.8320359 ,  0.04420534, -0.35048822,  0.6666084 ],
        [ 0.69449055, -0.38064197,  0.5204143 , -0.9067357 ,  0.08135187],
        [ 0.34750026, -0.9946707 ,  0.13325608, -0.48520643,  0.00784478],
        [ 0.04665023, -0.11869799,  0.06540714,  0.03166735, -0.03685782],
        [-0.7637425 , -0.18330225, -0.1342704 , -0.22876924, -0.6698236 ],
        [-0.06361876,  0.22113003,  0.5706431 , -0.59720993, -0.263737  ],
        [-0.02164517, -0.12007421, -0.40791076, -0.8447536 , -0.47136036],
        [-1.3762275 , -0.02534995,  0.23357524, -0.14083318,  0.0298421 ],
        [ 0.7895214 , -0.3284756 , -0.8241457 , -0.5841204 ,  0.18694809],
        [-0.46001127,  0.04347777,  0.94150555, -0.04406102,  0.845491  ],
        [-0.5934397 ,  0.891223  , -0.07054701, -0.15037079,  0.6251079 ],
        [-0.07849779,  0.06309937, -0.23583801, -0.26852795,  0.2579096 ],
        [-0.7977668 ,  0.

In [51]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure1.h5')

In [52]:
model_automobile, history_automobile=NN_model_structure_regression_2(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0071 - val_loss: 0.0052
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0066 - val_loss: 0.0053
Epoch 71/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0067 - val_loss: 0.0054
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0067 - val_loss: 0.0056
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0051
Epoch 74/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0049
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0067 - val_loss: 0.0048
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0054
Epoch 77/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0066 - val_loss: 0.0050
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 105us/step - loss: 0.0047 - val_loss: 0.0043
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0044
Epoch 225/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0046 - val_loss: 0.0042
Epoch 226/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0046 - val_loss: 0.0046
Epoch 227/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0047 - val_loss: 0.0049
Epoch 228/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0047 - val_loss: 0.0042
Epoch 229/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0047 - val_loss: 0.0040
Epoch 230/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0048 - val_loss: 0.0055
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0040
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 114us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 378/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0038
Epoch 379/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0039 - val_loss: 0.0037
Epoch 380/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0038
Epoch 381/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0041 - val_loss: 0.0041
Epoch 382/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0041 - val_loss: 0.0036
Epoch 383/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0044
Epoch 384/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0041 - val_loss: 0.0038
Epoch 385/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0040
Epoch 532/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0034
Epoch 533/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0035
Epoch 534/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0029 - val_loss: 0.0035
Epoch 535/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0036
Epoch 536/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0030 - val_loss: 0.0039
Epoch 537/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0029 - val_loss: 0.0032
Epoch 538/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0029 - val_loss: 0.0044
Epoch 539/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0031 - val_loss: 0.0033
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0033
Epoch 686/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 687/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0045
Epoch 688/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0046
Epoch 689/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0030 - val_loss: 0.0037
Epoch 690/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 691/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0027 - val_loss: 0.0039
Epoch 692/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0034
Epoch 693/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 133us/step - loss: 0.0021 - val_loss: 0.0037
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0038
Epoch 841/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 842/1000
143/143 [==============================] - 0s 130us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 843/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 844/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0040
Epoch 845/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0020 - val_loss: 0.0037
Epoch 846/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0035
Epoch 847/1000
143/143 [==============================] - 0s 127us/step - loss: 0.0025 - val_loss: 0.0048
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0040
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0049
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0037
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0055
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0042
Epoch 998/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0018 - val_loss: 0.0037
Epoch 999/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0019 - val_loss: 0.0048
Epoch 1000/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0042
0.009654373861849308


[array([[ 2.96538621e-01,  3.02006215e-01, -2.16691002e-01,
          8.51159155e-01,  2.78918654e-01],
        [-3.67945395e-02,  1.12654485e-01,  3.83709610e-01,
         -4.79814082e-01, -2.69231826e-01],
        [-1.06634343e+00,  3.07257921e-01, -4.36545968e-01,
         -9.36583281e-01, -5.88371575e-01],
        [-3.27710956e-01,  5.15551984e-01,  2.30621502e-01,
         -3.17301452e-01,  6.68447495e-01],
        [-1.63873002e-01, -1.00535996e-01,  6.91782981e-02,
         -4.50444221e-01,  3.58819246e-01],
        [-3.87287706e-01, -1.64963275e-01, -8.99864852e-01,
          9.65856850e-01, -1.93739146e-01],
        [ 6.40089344e-03,  6.50404513e-01,  3.40497606e-02,
         -2.30683327e-01, -3.60969901e-01],
        [ 3.90403122e-01,  4.14002448e-01, -6.75896928e-02,
          1.62398770e-01, -4.61649567e-01],
        [-6.35486245e-01,  4.36819196e-02, -6.68296516e-01,
          1.58710063e-01, -2.89600104e-01],
        [ 9.85796809e-01,  4.20030832e-01,  8.32027733e-01,
    

In [53]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure2.h5')

In [54]:
model_automobile, history_automobile=NN_model_structure_regression_3(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_30 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 139us/step - loss: 0.0067 - val_loss: 0.0058
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0056 - val_loss: 0.0038
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0057 - val_loss: 0.0055
Epoch 72/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0036
Epoch 73/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0062 - val_loss: 0.0050
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0053 - val_loss: 0.0037
Epoch 75/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0049
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0052 - val_loss: 0.0035
Epoch 77/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0052 - val_loss: 0.0041
Epoch 78/1000
143/143 [==============================] - 0s 126us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0050
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0054
Epoch 225/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0031 - val_loss: 0.0045
Epoch 226/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0032 - val_loss: 0.0063
Epoch 227/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0033 - val_loss: 0.0047
Epoch 228/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0046
Epoch 229/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0061
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0055
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0029 - val_loss: 0.0050
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0022 - val_loss: 0.0061
Epoch 378/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0021 - val_loss: 0.0058
Epoch 379/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0060
Epoch 380/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0055
Epoch 381/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0021 - val_loss: 0.0057
Epoch 382/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0077
Epoch 383/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0054
Epoch 384/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0023 - val_loss: 0.0053
Epoch 385/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0021 - val_loss: 0.0056
Epoch 386/1000
143/143 [==============================] - 0s 

Epoch 531/1000
143/143 [==============================] - 0s 136us/step - loss: 0.0018 - val_loss: 0.0054
Epoch 532/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0023 - val_loss: 0.0065
Epoch 533/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0021 - val_loss: 0.0060
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0055
Epoch 535/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0020 - val_loss: 0.0059
Epoch 536/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0059
Epoch 537/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0021 - val_loss: 0.0057
Epoch 538/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0064
Epoch 539/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0050
Epoch 540/1000
143/143 [======================

143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0067
Epoch 686/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0046
Epoch 687/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0023 - val_loss: 0.0070
Epoch 688/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0058
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0052
Epoch 690/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 691/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0052
Epoch 692/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0067
Epoch 693/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0022 - val_loss: 0.0046
Epoch 694/1000
143/143 [==============================] - ETA

143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0062
Epoch 840/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0021 - val_loss: 0.0065
Epoch 841/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0024 - val_loss: 0.0060
Epoch 842/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0055
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0018 - val_loss: 0.0053
Epoch 844/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0019 - val_loss: 0.0060
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0051
Epoch 846/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0015 - val_loss: 0.0066
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0051
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 120us/step - loss: 0.0015 - val_loss: 0.0054
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0054
Epoch 995/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0049
Epoch 996/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 997/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0065
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0015 - val_loss: 0.0048
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0047
Epoch 1000/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0057
0.015034993179142475


[array([[-0.792388  ,  0.8352976 ,  0.56654626, -0.05322287, -0.30493784],
        [-0.5113226 ,  0.305184  ,  0.15142906,  0.8989809 ,  0.8401689 ],
        [ 1.4343079 , -0.41060036,  0.1935165 ,  0.02391584,  0.60686857],
        [ 0.6301367 ,  0.3707191 ,  0.05371154,  0.00745301, -0.61123866],
        [ 0.42221132, -0.41640517, -0.0160974 , -0.38436106, -0.07389599],
        [-1.5815036 ,  0.79170394,  0.6203946 ,  0.8198848 ,  0.5863235 ],
        [ 0.7325629 ,  0.81557465, -0.12781943,  0.1914464 , -0.64791185],
        [ 0.41655102, -0.11018033,  0.41411   , -0.15567775, -0.5590992 ],
        [ 0.41913128, -0.53998387,  0.12068506, -0.5051021 , -0.873651  ],
        [-0.8225282 ,  0.78305614,  0.41209444,  0.9779791 , -0.13946828],
        [ 0.20452589, -0.58384603,  0.39220715,  0.16406694, -0.05798318],
        [-0.37285885, -0.5241034 , -0.30242723,  0.2247544 ,  0.68075496],
        [-0.37110952, -0.20133552,  0.16587389, -0.8218622 , -0.01654555],
        [-1.8740476 , -2.

In [55]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure3.h5')

In [56]:
model_automobile, history_automobile=NN_model_structure_regression_4(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_31 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_32 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0042
Epoch 70/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0052 - val_loss: 0.0037
Epoch 71/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0044
Epoch 72/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0053 - val_loss: 0.0037
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0054 - val_loss: 0.0041
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0050 - val_loss: 0.0040
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0036
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0051 - val_loss: 0.0038
Epoch 77/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0057 - val_loss: 0.0072
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0041
Epoch 224/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0030 - val_loss: 0.0048
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0036
Epoch 226/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0027 - val_loss: 0.0036
Epoch 227/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0029 - val_loss: 0.0060
Epoch 228/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0031 - val_loss: 0.0038
Epoch 229/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0036
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0031 - val_loss: 0.0061
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0043
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0065
Epoch 378/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0021 - val_loss: 0.0049
Epoch 379/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0021 - val_loss: 0.0047
Epoch 380/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0050
Epoch 381/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0065
Epoch 382/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0052
Epoch 383/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0051
Epoch 384/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0057
Epoch 385/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0056
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 107us/step - loss: 0.0018 - val_loss: 0.0058
Epoch 532/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0079
Epoch 533/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0061
Epoch 534/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0052
Epoch 535/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0065
Epoch 536/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0020 - val_loss: 0.0067
Epoch 537/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0055
Epoch 538/1000
143/143 [==============================] - 0s 130us/step - loss: 0.0017 - val_loss: 0.0054
Epoch 539/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0018 - val_loss: 0.0058
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0052
Epoch 686/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0059
Epoch 687/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0076
Epoch 688/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0018 - val_loss: 0.0055
Epoch 689/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0018 - val_loss: 0.0057
Epoch 690/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0017 - val_loss: 0.0070
Epoch 691/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0017 - val_loss: 0.0054
Epoch 692/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0056
Epoch 693/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0058
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0054
Epoch 840/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0013 - val_loss: 0.0061
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 842/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0016 - val_loss: 0.0054
Epoch 843/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0015 - val_loss: 0.0053
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0063
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0055
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0054
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0057
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 115us/step - loss: 0.0014 - val_loss: 0.0058
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 996/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0013 - val_loss: 0.0050
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0047
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0052
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0057
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0054
0.0064208111725747585


[array([[ 7.20673800e-01, -5.06255329e-01, -1.71767280e-01,
         -2.53437340e-01,  9.21964124e-02,  6.05357766e-01,
          8.68049264e-02,  2.23658830e-01, -6.06232822e-01,
         -3.03710610e-01],
        [-4.26197141e-01,  2.68055677e-01,  1.62956104e-01,
         -1.52625307e-01,  4.64951724e-01, -6.59884810e-01,
         -7.22933412e-02, -4.42152709e-01, -1.58244327e-01,
         -1.90732926e-01],
        [-2.88230330e-01, -1.68657407e-01, -9.25236568e-02,
         -5.78952581e-02, -3.74478161e-01, -7.91995108e-01,
         -1.71004802e-01, -3.30248624e-01,  4.02846813e-01,
         -1.82194095e-02],
        [ 1.10337257e+00,  9.98816043e-02,  1.64170414e-01,
         -4.17907566e-01,  2.02510357e-01, -6.19491696e-01,
         -2.71853536e-01,  8.97835553e-01, -4.48642552e-01,
         -4.38523561e-01],
        [ 7.23150969e-02,  2.54875813e-02, -1.35496045e-02,
         -1.85862899e-01,  2.07694024e-01, -1.33636579e-01,
         -8.52980912e-02, -8.30966160e-02, -1.438722

In [57]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure4.h5')

In [58]:
model_automobile, history_automobile=NN_model_structure_regression_5(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_33 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 126us/step - loss: 0.0061 - val_loss: 0.0044
Epoch 70/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0064 - val_loss: 0.0047
Epoch 71/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0066 - val_loss: 0.0048
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0053
Epoch 73/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0060 - val_loss: 0.0046
Epoch 74/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0066 - val_loss: 0.0057
Epoch 75/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0063 - val_loss: 0.0053
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0066 - val_loss: 0.0050
Epoch 77/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 78/1000
143/143 [==============================] - 0s 126us/ste

143/143 [==============================] - 0s 118us/step - loss: 0.0036 - val_loss: 0.0044
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0039
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0040
Epoch 226/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0039
Epoch 227/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0041
Epoch 228/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0042
Epoch 230/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0032 - val_loss: 0.0056
Epoch 231/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0034 - val_loss: 0.0039
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0032
Epoch 378/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 379/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0043
Epoch 380/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0036
Epoch 381/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0025 - val_loss: 0.0038
Epoch 382/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0039
Epoch 383/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 384/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0043
Epoch 385/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0065
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 139us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 532/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0036
Epoch 533/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0026 - val_loss: 0.0039
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0060
Epoch 535/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0038
Epoch 536/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0019 - val_loss: 0.0035
Epoch 537/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0038
Epoch 538/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0031
Epoch 539/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0038
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0037
Epoch 686/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0035
Epoch 687/1000
143/143 [==============================] - 0s 128us/step - loss: 0.0013 - val_loss: 0.0037
Epoch 688/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0050
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0045
Epoch 690/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0034
Epoch 691/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0035
Epoch 692/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0013 - val_loss: 0.0037
Epoch 693/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0037
Epoch 694/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0039
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0041
Epoch 841/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0011 - val_loss: 0.0042
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 843/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0040
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0012 - val_loss: 0.0049
Epoch 846/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 847/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0042
Epoch 994/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0012 - val_loss: 0.0049
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 9.8161e-04 - val_loss: 0.0041
Epoch 996/1000
143/143 [==============================] - 0s 118us/step - loss: 9.8963e-04 - val_loss: 0.0049
Epoch 997/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0051
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 999/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 1000/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0012 - val_loss: 0.0039
0.009712672792375088


[array([[ 3.0819517e-01,  1.9252868e-01,  2.4932690e-01, -1.3983034e-02,
         -8.9831609e-01,  1.7871407e-01, -4.3583980e-01,  1.7268035e-01,
          1.9031966e-01,  3.5769415e-01],
        [-2.3768518e-02, -5.0292236e-01,  4.5423552e-01,  6.1553615e-01,
         -2.8360093e-01, -3.5647106e-01, -5.1185107e-01, -8.3151735e-02,
         -3.8281971e-01,  1.9003157e-02],
        [-2.0098183e-01,  1.9330713e-01, -5.2858835e-01,  2.6283902e-01,
          9.6761936e-01, -5.5107337e-01,  4.2783856e-01, -1.5716547e-01,
          8.7647066e-02, -3.3642927e-01],
        [ 6.7926002e-01, -2.9228437e-01,  8.2343715e-01,  7.4097723e-01,
          7.9637098e-01,  6.8369053e-02,  1.4360452e-02,  6.3393301e-01,
         -8.3496355e-02,  5.3214240e-01],
        [-1.7533982e-01,  2.4053140e-01, -8.7647319e-01,  3.8572270e-01,
          1.6414054e-01,  8.7323773e-01,  2.2078288e-01, -4.7912183e-01,
          7.2972678e-02, -6.5828055e-01],
        [ 3.1930082e-02,  5.0202620e-01, -4.3037954e-01,  5.

In [59]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure5.h5')

In [60]:
model_automobile, history_automobile=NN_model_structure_regression_6(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000)
model_automobile.get_weights()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_35 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_36 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 16        
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 112us/step - loss: 0.0078 - val_loss: 0.0049
Epoch 70/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0073 - val_loss: 0.0051
Epoch 71/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0069 - val_loss: 0.0050
Epoch 72/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0069 - val_loss: 0.0047
Epoch 73/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0069 - val_loss: 0.0048
Epoch 74/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0069 - val_loss: 0.0056
Epoch 75/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0071 - val_loss: 0.0049
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0067 - val_loss: 0.0061
Epoch 77/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0071 - val_loss: 0.0049
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0041
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0042 - val_loss: 0.0040
Epoch 225/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0046 - val_loss: 0.0041
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0041 - val_loss: 0.0050
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 228/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0040 - val_loss: 0.0049
Epoch 229/1000
143/143 [==============================] - 0s 130us/step - loss: 0.0040 - val_loss: 0.0046
Epoch 230/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0050 - val_loss: 0.0042
Epoch 231/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0039 - val_loss: 0.0041
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 116us/step - loss: 0.0026 - val_loss: 0.0043
Epoch 378/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0042
Epoch 379/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0025 - val_loss: 0.0038
Epoch 380/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0024 - val_loss: 0.0045
Epoch 381/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0042
Epoch 382/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0022 - val_loss: 0.0039
Epoch 383/1000
143/143 [==============================] - 0s 129us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 384/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0024 - val_loss: 0.0043
Epoch 385/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0021 - val_loss: 0.0045
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 110us/step - loss: 0.0029 - val_loss: 0.0043
Epoch 532/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0031
Epoch 533/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0019 - val_loss: 0.0044
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0038
Epoch 535/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 536/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0016 - val_loss: 0.0040
Epoch 537/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 538/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0022 - val_loss: 0.0047
Epoch 539/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0021 - val_loss: 0.0055
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 686/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0052
Epoch 687/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0018 - val_loss: 0.0043
Epoch 688/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0047
Epoch 690/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 691/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0039
Epoch 692/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0046
Epoch 693/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0054
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0044
Epoch 840/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 841/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0011 - val_loss: 0.0047
Epoch 842/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0049
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0050
Epoch 846/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0043
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 995/1000
143/143 [==============================] - 0s 109us/step - loss: 9.3383e-04 - val_loss: 0.0041
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 9.4684e-04 - val_loss: 0.0042
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 9.8327e-04 - val_loss: 0.0039
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 8.9740e-04 - val_loss: 0.0043
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 9.6440e-04 - val_loss: 0.0039
Epoch 1000/1000
143/143 [==============================] - 0s 111us/step - loss: 9.5475e-04 - val_loss: 0.0039
0.007109248079359531


[array([[ 0.26568326,  0.4342506 ,  0.36667058,  0.022689  , -0.33124462,
          0.4357042 ,  0.9735705 ,  0.25583595,  0.405627  ,  0.8572842 ],
        [ 0.07279145, -0.06734834, -0.7621755 , -0.46524894, -0.04930552,
         -0.8873211 ,  0.4001797 , -0.36227265,  0.7401857 ,  0.30332628],
        [-0.16432795, -0.2833461 , -0.03923907,  0.24485503,  0.1259899 ,
         -0.43068945, -0.16862904,  0.3329162 , -0.49666277,  0.12314925],
        [ 1.0148062 ,  0.01562084, -0.38034132,  0.05781981,  0.17653336,
          0.22948806,  0.9483628 ,  0.10622626, -0.69160664, -1.0884848 ],
        [ 0.4539218 , -0.29670238,  0.4275147 , -0.58270025, -0.7091275 ,
          0.05065642,  0.02670469,  0.25055048,  0.55591536, -0.05906231],
        [ 0.17577617,  0.01053759,  0.11207725, -0.47418842,  0.3137868 ,
          0.30382705,  0.49177256, -0.03913935,  0.6670395 ,  0.05760032],
        [ 0.40890336, -0.00532622, -1.3990468 , -0.27278847,  0.07344352,
         -0.6309691 , -0.0934196

In [61]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure6.h5')

In [62]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_1(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_37 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_38 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 6         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 35.75

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0316 - val_loss: 0.0107
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0308 - val_loss: 0.0113
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0303 - val_loss: 0.0122
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0299 - val_loss: 0.0130
Epoch 75/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0293 - val_loss: 0.0134
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0286 - val_loss: 0.0135
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0276 - val_loss: 0.0131
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0264 - val_loss: 0.0128
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0253 - val_loss: 0.0126
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0105 - val_loss: 0.0034
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0104 - val_loss: 0.0033
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0103 - val_loss: 0.0033
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0102 - val_loss: 0.0033
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0101 - val_loss: 0.0032
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0101 - val_loss: 0.0032
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0100 - val_loss: 0.0031
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0099 - val_loss: 0.0031
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0098 - val_loss: 0.0031
Epoch 160/1000
32/32 [==============================] - 0s 123us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0017
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0059 - val_loss: 0.0017
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0059 - val_loss: 0.0017
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0058 - val_loss: 0.0017
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0058 - val_loss: 0.0017
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0058 - val_loss: 0.0016
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0057 - val_loss: 0.0016
Epoch 237/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0057 - val_loss: 0.0016
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0057 - val_loss: 0.0016
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 9.9969e-04
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 9.9355e-04
Epoch 311/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0038 - val_loss: 9.8748e-04
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 9.8144e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 9.7545e-04
Epoch 314/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 9.6950e-04
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 9.6363e-04
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 9.5780e-04
Epoch 317/1000
32/32 [==============================] - 0s 134us/step - loss: 0.0037 - val_loss: 9.5200e-04
Epoch 318/1000
32/32 [======

...

Epoch 765/1000
32/32 [==============================] - 0s 188us/step - loss: 9.7091e-04 - val_loss: 2.9273e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6903e-04 - val_loss: 2.9248e-04
Epoch 767/1000
32/32 [==============================] - 0s 125us/step - loss: 9.6716e-04 - val_loss: 2.9222e-04
Epoch 768/1000
32/32 [==============================] - 0s 218us/step - loss: 9.6529e-04 - val_loss: 2.9197e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6343e-04 - val_loss: 2.9172e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 9.6157e-04 - val_loss: 2.9148e-04
Epoch 771/1000
32/32 [==============================] - 0s 183us/step - loss: 9.5972e-04 - val_loss: 2.9123e-04
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 9.5787e-04 - val_loss: 2.9098e-04
Epoch 773/1000
32/32 [==============================] - 0s 218us/step - loss: 9.5603e-04 - val_loss: 2.9

Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4568e-04 - val_loss: 2.7530e-04
Epoch 840/1000
32/32 [==============================] - 0s 218us/step - loss: 8.4416e-04 - val_loss: 2.7507e-04
Epoch 841/1000
32/32 [==============================] - 0s 124us/step - loss: 8.4265e-04 - val_loss: 2.7485e-04
Epoch 842/1000
32/32 [==============================] - 0s 156us/step - loss: 8.4114e-04 - val_loss: 2.7463e-04
Epoch 843/1000
32/32 [==============================] - 0s 187us/step - loss: 8.3962e-04 - val_loss: 2.7441e-04
Epoch 844/1000
32/32 [==============================] - 0s 125us/step - loss: 8.3813e-04 - val_loss: 2.7419e-04
Epoch 845/1000
32/32 [==============================] - 0s 156us/step - loss: 8.3663e-04 - val_loss: 2.7397e-04
Epoch 846/1000
32/32 [==============================] - 0s 156us/step - loss: 8.3514e-04 - val_loss: 2.7377e-04
Epoch 847/1000
32/32 [==============================] - 0s 125us/step - loss: 8.3364e-04 - val_loss: 2.7

Epoch 913/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4334e-04 - val_loss: 2.5963e-04
Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4208e-04 - val_loss: 2.5943e-04
Epoch 915/1000
32/32 [==============================] - 0s 155us/step - loss: 7.4082e-04 - val_loss: 2.5922e-04
Epoch 916/1000
32/32 [==============================] - 0s 125us/step - loss: 7.3957e-04 - val_loss: 2.5902e-04
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3833e-04 - val_loss: 2.5882e-04
Epoch 918/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3709e-04 - val_loss: 2.5861e-04
Epoch 919/1000
32/32 [==============================] - 0s 125us/step - loss: 7.3584e-04 - val_loss: 2.5840e-04
Epoch 920/1000
32/32 [==============================] - 0s 155us/step - loss: 7.3460e-04 - val_loss: 2.5819e-04
Epoch 921/1000
32/32 [==============================] - 0s 187us/step - loss: 7.3337e-04 - val_loss: 2.5

Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5779e-04 - val_loss: 2.4475e-04
Epoch 988/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5672e-04 - val_loss: 2.4454e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5566e-04 - val_loss: 2.4435e-04
Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5461e-04 - val_loss: 2.4414e-04
Epoch 991/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5355e-04 - val_loss: 2.4396e-04
Epoch 992/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5250e-04 - val_loss: 2.4377e-04
Epoch 993/1000
32/32 [==============================] - 0s 187us/step - loss: 6.5145e-04 - val_loss: 2.4357e-04
Epoch 994/1000
32/32 [==============================] - 0s 124us/step - loss: 6.5041e-04 - val_loss: 2.4337e-04
Epoch 995/1000
32/32 [==============================] - 0s 125us/step - loss: 6.4936e-04 - val_loss: 2.4

[array([[-0.54972893, -0.678983  ,  0.63555014,  1.0720394 ,  0.40174437],
        [ 0.8432477 , -0.3559134 , -0.84869003,  0.00250493, -0.81218654],
        [ 1.0256863 , -1.6136305 ,  1.1682248 ,  0.6581044 ,  1.0225453 ]],
       dtype=float32),
 array([-0.9119816 , -0.58590555, -0.7766057 ,  0.75616705, -0.90602237],
       dtype=float32),
 array([[ 0.2402422 , -0.9074045 , -0.48534715, -0.6994538 , -0.5002308 ],
        [-0.8628928 , -0.5048205 , -0.7996956 , -0.5149932 , -0.60349137],
        [ 0.5182076 , -0.5165274 , -0.60075426, -0.9451504 , -0.33365694],
        [ 0.01527457, -0.46545038,  0.5315274 ,  0.05733462, -0.73718655],
        [ 0.6223832 , -0.24265811,  0.57777816, -0.48810044, -0.39313772]],
       dtype=float32),
 array([1.0144831 , 1.0199761 , 1.0346524 , 1.0254389 , 0.59769267],
       dtype=float32),
 array([[0.48527893],
        [0.98558635],
        [0.5632267 ],
        [1.0210392 ],
        [0.15988076]], dtype=float32),
 array([1.003348], dtype=float32)]

In [63]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure1.h5')

In [64]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_2(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_39 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_40 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 36.58

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0143 - val_loss: 0.0093
Epoch 72/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0149 - val_loss: 0.0087
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0146 - val_loss: 0.0072
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0134 - val_loss: 0.0054
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0117 - val_loss: 0.0036
Epoch 76/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0099 - val_loss: 0.0021
Epoch 77/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0084 - val_loss: 0.0012
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0075 - val_loss: 7.0427e-04
Epoch 79/1000
32/32 [==============================] - 0s 137us/step - loss: 0.0072 - val_loss: 6.0947e-04
Epoch 80/1000
32/32 [==============================] - 0s 125us/s

Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1219e-04 - val_loss: 8.6425e-05
Epoch 149/1000
32/32 [==============================] - 0s 199us/step - loss: 4.9743e-04 - val_loss: 8.6929e-05
Epoch 150/1000
32/32 [==============================] - 0s 165us/step - loss: 4.8374e-04 - val_loss: 8.7117e-05
Epoch 151/1000
32/32 [==============================] - 0s 126us/step - loss: 4.7094e-04 - val_loss: 8.6641e-05
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 4.5880e-04 - val_loss: 8.5313e-05
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4712e-04 - val_loss: 8.3105e-05
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3575e-04 - val_loss: 8.0178e-05
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2468e-04 - val_loss: 7.6822e-05
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1397e-04 - val_loss: 7.3

Epoch 222/1000
32/32 [==============================] - 0s 187us/step - loss: 2.0325e-04 - val_loss: 8.5897e-05
Epoch 223/1000
32/32 [==============================] - 0s 124us/step - loss: 2.0266e-04 - val_loss: 8.6130e-05
Epoch 224/1000
32/32 [==============================] - 0s 124us/step - loss: 2.0208e-04 - val_loss: 8.6362e-05
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0152e-04 - val_loss: 8.6575e-05
Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 2.0097e-04 - val_loss: 8.6770e-05
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 2.0044e-04 - val_loss: 8.6950e-05
Epoch 228/1000
32/32 [==============================] - 0s 155us/step - loss: 1.9992e-04 - val_loss: 8.7114e-05
Epoch 229/1000
32/32 [==============================] - 0s 154us/step - loss: 1.9942e-04 - val_loss: 8.7268e-05
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9893e-04 - val_loss: 8.7

Epoch 296/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7979e-04 - val_loss: 8.8012e-05
Epoch 297/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7959e-04 - val_loss: 8.7959e-05
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7939e-04 - val_loss: 8.7888e-05
Epoch 299/1000
32/32 [==============================] - 0s 125us/step - loss: 1.7919e-04 - val_loss: 8.7831e-05
Epoch 300/1000
32/32 [==============================] - 0s 187us/step - loss: 1.7899e-04 - val_loss: 8.7762e-05
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7880e-04 - val_loss: 8.7701e-05
Epoch 302/1000
32/32 [==============================] - 0s 124us/step - loss: 1.7860e-04 - val_loss: 8.7640e-05
Epoch 303/1000
32/32 [==============================] - 0s 186us/step - loss: 1.7841e-04 - val_loss: 8.7573e-05
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 1.7821e-04 - val_loss: 8.7

...

Epoch 740/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2973e-04 - val_loss: 5.9461e-05
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2965e-04 - val_loss: 5.9412e-05
Epoch 742/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2956e-04 - val_loss: 5.9363e-05
Epoch 743/1000
32/32 [==============================] - 0s 137us/step - loss: 1.2948e-04 - val_loss: 5.9312e-05
Epoch 744/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2939e-04 - val_loss: 5.9267e-05
Epoch 745/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2931e-04 - val_loss: 5.9215e-05
Epoch 746/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2923e-04 - val_loss: 5.9167e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2914e-04 - val_loss: 5.9114e-05
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2906e-04 - val_loss: 5.9

Epoch 814/1000
32/32 [==============================] - 0s 187us/step - loss: 1.2364e-04 - val_loss: 5.5922e-05
Epoch 815/1000
32/32 [==============================] - 0s 133us/step - loss: 1.2356e-04 - val_loss: 5.5873e-05
Epoch 816/1000
32/32 [==============================] - 0s 116us/step - loss: 1.2347e-04 - val_loss: 5.5825e-05
Epoch 817/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2339e-04 - val_loss: 5.5789e-05
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2331e-04 - val_loss: 5.5739e-05
Epoch 819/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2323e-04 - val_loss: 5.5695e-05
Epoch 820/1000
32/32 [==============================] - 0s 187us/step - loss: 1.2315e-04 - val_loss: 5.5645e-05
Epoch 821/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2307e-04 - val_loss: 5.5603e-05
Epoch 822/1000
32/32 [==============================] - 0s 124us/step - loss: 1.2299e-04 - val_loss: 5.5

Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1777e-04 - val_loss: 5.2682e-05
Epoch 889/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1769e-04 - val_loss: 5.2647e-05
Epoch 890/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1762e-04 - val_loss: 5.2604e-05
Epoch 891/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1754e-04 - val_loss: 5.2562e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1746e-04 - val_loss: 5.2517e-05
Epoch 893/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1738e-04 - val_loss: 5.2472e-05
Epoch 894/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1731e-04 - val_loss: 5.2438e-05
Epoch 895/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1723e-04 - val_loss: 5.2390e-05
Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1715e-04 - val_loss: 5.2

Epoch 962/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1212e-04 - val_loss: 4.9701e-05
Epoch 963/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1205e-04 - val_loss: 4.9664e-05
Epoch 964/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1197e-04 - val_loss: 4.9628e-05
Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1190e-04 - val_loss: 4.9588e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1182e-04 - val_loss: 4.9551e-05
Epoch 967/1000
32/32 [==============================] - 0s 125us/step - loss: 1.1175e-04 - val_loss: 4.9508e-05
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1167e-04 - val_loss: 4.9467e-05
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1160e-04 - val_loss: 4.9435e-05
Epoch 970/1000
32/32 [==============================] - 0s 155us/step - loss: 1.1152e-04 - val_loss: 4.9

[array([[-1.1731365 , -1.0928568 , -0.65612894, -0.4138755 ,  0.71974915],
        [ 0.08791504,  0.27462038, -0.03303109,  0.06003816,  0.52427894],
        [ 1.2347534 , -0.51848215, -0.9254643 ,  0.10201246,  0.53958184]],
       dtype=float32),
 array([-0.40893486,  0.6113161 , -0.5411161 ,  0.2468039 , -0.04456357],
       dtype=float32),
 array([[ 0.56758225, -0.08212385, -0.1566527 ,  0.10075039,  0.21810928,
          0.33117977, -0.16311736, -0.6014744 , -0.68491024,  0.8506684 ],
        [-0.5882047 , -0.7475559 , -0.45515734,  0.04794963,  0.3319468 ,
         -0.06464512,  0.10279346,  0.23261534,  0.42378843, -0.47234806],
        [ 0.14885715, -0.1511121 ,  0.50857896,  0.38196126,  0.24780093,
         -0.8838558 , -0.3084874 , -0.04967599,  0.48106587,  0.81422985],
        [ 0.06306236,  0.33022505,  0.32202426, -0.94480294,  0.59045815,
          0.3635264 , -0.2439131 ,  0.2627529 ,  0.6648691 , -0.21039443],
        [ 0.17621878,  0.04984105, -0.26300627,  0.6358560

In [65]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure2.h5')

In [66]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_3(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_62 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_42 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 16        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0183 - val_loss: 0.0070
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0176 - val_loss: 0.0063
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0157 - val_loss: 0.0054
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0133 - val_loss: 0.0050
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0115 - val_loss: 0.0054
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0106 - val_loss: 0.0066
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0106 - val_loss: 0.0079
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0109 - val_loss: 0.0091
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0113 - val_loss: 0.0097
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0049 - val_loss: 9.2657e-04
Epoch 151/1000
32/32 [==============================] - 0s 142us/step - loss: 0.0048 - val_loss: 9.1403e-04
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0048 - val_loss: 9.0487e-04
Epoch 153/1000
32/32 [==============================] - 0s 132us/step - loss: 0.0047 - val_loss: 8.9870e-04
Epoch 154/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0047 - val_loss: 8.9466e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 8.9130e-04
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0046 - val_loss: 8.8707e-04
Epoch 157/1000
32/32 [==============================] - 0s 131us/step - loss: 0.0046 - val_loss: 8.8074e-04
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 8.7165e-04
Epoch 159/1000
32/32 [======

Epoch 226/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 6.3351e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 6.3581e-04
Epoch 228/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 6.3826e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 6.4084e-04
Epoch 230/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0026 - val_loss: 6.4351e-04
Epoch 231/1000
32/32 [==============================] - 0s 133us/step - loss: 0.0026 - val_loss: 6.4631e-04
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 6.4924e-04
Epoch 233/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0026 - val_loss: 6.5230e-04
Epoch 234/1000
32/32 [==============================] - 0s 144us/step - loss: 0.0026 - val_loss: 6.5544e-04
Epoch 235/1000
32/32 [======

Epoch 302/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 9.0508e-04
Epoch 303/1000
32/32 [==============================] - 0s 129us/step - loss: 0.0018 - val_loss: 9.0770e-04
Epoch 304/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0018 - val_loss: 9.1024e-04
Epoch 305/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0018 - val_loss: 9.1273e-04
Epoch 306/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0017 - val_loss: 9.1517e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 9.1758e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 9.1990e-04
Epoch 309/1000
32/32 [==============================] - 0s 149us/step - loss: 0.0017 - val_loss: 9.2216e-04
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 9.2438e-04
Epoch 311/1000
32/32 [======

...

Epoch 750/1000
32/32 [==============================] - 0s 124us/step - loss: 4.2202e-04 - val_loss: 6.2651e-04
Epoch 751/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2103e-04 - val_loss: 6.2585e-04
Epoch 752/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2001e-04 - val_loss: 6.2519e-04
Epoch 753/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1902e-04 - val_loss: 6.2453e-04
Epoch 754/1000
32/32 [==============================] - 0s 186us/step - loss: 4.1801e-04 - val_loss: 6.2389e-04
Epoch 755/1000
32/32 [==============================] - 0s 126us/step - loss: 4.1702e-04 - val_loss: 6.2323e-04
Epoch 756/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1603e-04 - val_loss: 6.2261e-04
Epoch 757/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1505e-04 - val_loss: 6.2196e-04
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1406e-04 - val_loss: 6.2

Epoch 824/1000
32/32 [==============================] - 0s 155us/step - loss: 3.5697e-04 - val_loss: 5.8224e-04
Epoch 825/1000
32/32 [==============================] - 0s 124us/step - loss: 3.5622e-04 - val_loss: 5.8169e-04
Epoch 826/1000
32/32 [==============================] - 0s 155us/step - loss: 3.5546e-04 - val_loss: 5.8115e-04
Epoch 827/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5470e-04 - val_loss: 5.8061e-04
Epoch 828/1000
32/32 [==============================] - 0s 155us/step - loss: 3.5396e-04 - val_loss: 5.8008e-04
Epoch 829/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5322e-04 - val_loss: 5.7952e-04
Epoch 830/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5247e-04 - val_loss: 5.7899e-04
Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5173e-04 - val_loss: 5.7845e-04
Epoch 832/1000
32/32 [==============================] - 0s 149us/step - loss: 3.5099e-04 - val_loss: 5.7

Epoch 898/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0809e-04 - val_loss: 5.4535e-04
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0752e-04 - val_loss: 5.4490e-04
Epoch 900/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0696e-04 - val_loss: 5.4444e-04
Epoch 901/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0639e-04 - val_loss: 5.4398e-04
Epoch 902/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0582e-04 - val_loss: 5.4355e-04
Epoch 903/1000
32/32 [==============================] - 0s 186us/step - loss: 3.0526e-04 - val_loss: 5.4309e-04
Epoch 904/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0470e-04 - val_loss: 5.4263e-04
Epoch 905/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0415e-04 - val_loss: 5.4219e-04
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0359e-04 - val_loss: 5.4

Epoch 972/1000
32/32 [==============================] - 0s 187us/step - loss: 2.7116e-04 - val_loss: 5.1417e-04
Epoch 973/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7073e-04 - val_loss: 5.1377e-04
Epoch 974/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7030e-04 - val_loss: 5.1338e-04
Epoch 975/1000
32/32 [==============================] - 0s 187us/step - loss: 2.6987e-04 - val_loss: 5.1298e-04
Epoch 976/1000
32/32 [==============================] - 0s 124us/step - loss: 2.6944e-04 - val_loss: 5.1260e-04
Epoch 977/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6902e-04 - val_loss: 5.1221e-04
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6859e-04 - val_loss: 5.1183e-04
Epoch 979/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6817e-04 - val_loss: 5.1144e-04
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6774e-04 - val_loss: 5.1

[array([[ 1.1581059e+00,  6.3869971e-01,  7.7265894e-01,  1.2397592e+00,
         -6.4233893e-01],
        [-1.0345707e+00, -1.4063919e-01, -5.3761059e-01, -2.3501703e-01,
         -3.5349315e-01],
        [ 4.7792643e-01,  4.2390421e-01, -9.0355414e-01,  1.0373666e-03,
         -1.1366388e+00]], dtype=float32),
 array([-0.64040315, -0.44148642,  0.5906181 , -0.47602898, -0.58039093],
       dtype=float32),
 array([[-1.92478508e-01, -4.01316196e-01,  6.07602119e-01,
          8.09499741e-01,  3.58353317e-01,  7.25540891e-02,
          6.19982839e-01,  6.74844757e-02, -4.07430887e-01,
         -5.09212017e-01,  1.59765616e-01,  5.48962355e-01,
          2.87529469e-01,  2.87123352e-01, -5.05611636e-02],
        [-4.31572646e-01, -3.09531599e-01, -2.21672595e-01,
          1.39170483e-01, -1.19934820e-01,  3.94964814e-01,
          4.50524658e-01,  5.79960234e-02,  8.08351710e-02,
         -6.52391613e-02, -6.13993227e-01, -1.94198534e-01,
          1.30490154e-01,  1.21540479e-01, -1.11

In [67]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure3.h5')

In [68]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_4(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_43 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_44 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0226 - val_loss: 0.0056
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0239 - val_loss: 0.0051
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0253 - val_loss: 0.0048
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0260 - val_loss: 0.0045
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0255 - val_loss: 0.0041
Epoch 76/1000
32/32 [==============================] - 0s 182us/step - loss: 0.0241 - val_loss: 0.0039
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0221 - val_loss: 0.0041
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0201 - val_loss: 0.0049
Epoch 79/1000
32/32 [==============================] - 0s 177us/step - loss: 0.0186 - val_loss: 0.0062
Epoch 80/1000
32/32 [==============================] - 0s 155us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0076 - val_loss: 0.0055
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0075 - val_loss: 0.0055
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0074 - val_loss: 0.0054
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0074 - val_loss: 0.0053
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0073 - val_loss: 0.0053
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0072 - val_loss: 0.0052
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0072 - val_loss: 0.0052
Epoch 158/1000
32/32 [==============================] - 0s 148us/step - loss: 0.0071 - val_loss: 0.0051
Epoch 159/1000
32/32 [==============================] - 0s 144us/step - loss: 0.0071 - val_loss: 0.0051
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0039 - val_loss: 0.0033
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0039 - val_loss: 0.0033
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0039 - val_loss: 0.0032
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0039 - val_loss: 0.0032
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 0.0032
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 0.0032
Epoch 236/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0038 - val_loss: 0.0031
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0031
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0031
Epoch 239/1000
32/32 [==============================] - 0s 187us

Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 311/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 313/1000
32/32 [==============================] - 0s 152us/step - loss: 0.0023 - val_loss: 0.0020
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0022 - val_loss: 0.0020
Epoch 315/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0022 - val_loss: 0.0020
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 317/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 318/1000
32/32 [==============================] - 0s 155us

...

Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7847e-04 - val_loss: 5.8550e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7742e-04 - val_loss: 5.8436e-04
Epoch 767/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7638e-04 - val_loss: 5.8323e-04
Epoch 768/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7534e-04 - val_loss: 5.8209e-04
Epoch 769/1000
32/32 [==============================] - 0s 140us/step - loss: 4.7431e-04 - val_loss: 5.8097e-04
Epoch 770/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7327e-04 - val_loss: 5.7985e-04
Epoch 771/1000
32/32 [==============================] - 0s 187us/step - loss: 4.7224e-04 - val_loss: 5.7873e-04
Epoch 772/1000
32/32 [==============================] - 0s 124us/step - loss: 4.7122e-04 - val_loss: 5.7761e-04
Epoch 773/1000
32/32 [==============================] - 0s 155us/step - loss: 4.7020e-04 - val_loss: 5.7

Epoch 839/1000
32/32 [==============================] - 0s 155us/step - loss: 4.0990e-04 - val_loss: 5.0721e-04
Epoch 840/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0908e-04 - val_loss: 5.0624e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 4.0827e-04 - val_loss: 5.0525e-04
Epoch 842/1000
32/32 [==============================] - 0s 155us/step - loss: 4.0745e-04 - val_loss: 5.0428e-04
Epoch 843/1000
32/32 [==============================] - 0s 126us/step - loss: 4.0665e-04 - val_loss: 5.0331e-04
Epoch 844/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0584e-04 - val_loss: 5.0232e-04
Epoch 845/1000
32/32 [==============================] - 0s 187us/step - loss: 4.0504e-04 - val_loss: 5.0137e-04
Epoch 846/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0424e-04 - val_loss: 5.0039e-04
Epoch 847/1000
32/32 [==============================] - 0s 124us/step - loss: 4.0344e-04 - val_loss: 4.9

Epoch 913/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5596e-04 - val_loss: 4.3918e-04
Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5531e-04 - val_loss: 4.3833e-04
Epoch 915/1000
32/32 [==============================] - 0s 120us/step - loss: 3.5467e-04 - val_loss: 4.3747e-04
Epoch 916/1000
32/32 [==============================] - 0s 186us/step - loss: 3.5402e-04 - val_loss: 4.3665e-04
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5338e-04 - val_loss: 4.3579e-04
Epoch 918/1000
32/32 [==============================] - 0s 124us/step - loss: 3.5274e-04 - val_loss: 4.3494e-04
Epoch 919/1000
32/32 [==============================] - 0s 186us/step - loss: 3.5210e-04 - val_loss: 4.3408e-04
Epoch 920/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5146e-04 - val_loss: 4.3324e-04
Epoch 921/1000
32/32 [==============================] - 0s 124us/step - loss: 3.5083e-04 - val_loss: 4.3

Epoch 987/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1292e-04 - val_loss: 3.8002e-04
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1240e-04 - val_loss: 3.7928e-04
Epoch 989/1000
32/32 [==============================] - 0s 155us/step - loss: 3.1189e-04 - val_loss: 3.7853e-04
Epoch 990/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1137e-04 - val_loss: 3.7779e-04
Epoch 991/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1085e-04 - val_loss: 3.7705e-04
Epoch 992/1000
32/32 [==============================] - 0s 186us/step - loss: 3.1033e-04 - val_loss: 3.7632e-04
Epoch 993/1000
32/32 [==============================] - 0s 186us/step - loss: 3.0982e-04 - val_loss: 3.7559e-04
Epoch 994/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0932e-04 - val_loss: 3.7485e-04
Epoch 995/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0881e-04 - val_loss: 3.7

[array([[-0.12655033,  0.76124   , -0.1056475 ,  0.3738104 ,  0.73713744,
         -0.8957579 ,  0.41234156,  0.6047862 , -0.69958353,  1.1099483 ],
        [-0.7499556 , -0.2970074 , -0.50185335,  0.23400494, -0.42433923,
          0.21782087, -0.8553799 , -0.67974263,  0.4196289 ,  0.12673292],
        [-0.81490064,  0.7082258 , -1.5542495 ,  0.40232897, -0.46948665,
         -0.13122715,  0.43894076, -1.0009674 ,  0.5873346 ,  1.5751377 ]],
       dtype=float32),
 array([ 0.3975633 , -0.1501335 , -0.56929475,  0.57183945,  0.5495596 ,
         0.49859458, -0.56042725,  0.42125994, -0.08397902,  0.2904021 ],
       dtype=float32),
 array([[-0.8248251 ,  0.01927135,  0.1301535 ,  0.1875432 ,  0.3385841 ],
        [ 0.7041947 ,  0.37417302,  0.34721392,  0.3177513 , -0.82020533],
        [ 0.84026134, -0.7273104 ,  0.4327214 , -0.655121  , -0.8331828 ],
        [-0.48471057, -0.5806327 , -0.41717204,  0.07953556,  0.29877815],
        [-0.2433252 , -0.20676278, -0.26414007,  0.59842104

In [69]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure4.h5')

In [70]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_5(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_46 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 15ms/step - loss: 35.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0145 - val_loss: 0.0204
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0152 - val_loss: 0.0211
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0157 - val_loss: 0.0209
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0159 - val_loss: 0.0198
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0156 - val_loss: 0.0182
Epoch 76/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0149 - val_loss: 0.0163
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0141 - val_loss: 0.0144
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0132 - val_loss: 0.0128
Epoch 79/1000
32/32 [==============================] - 0s 175us/step - loss: 0.0124 - val_loss: 0.0117
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0050 - val_loss: 0.0052
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0050 - val_loss: 0.0051
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0050 - val_loss: 0.0051
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0049 - val_loss: 0.0050
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0049 - val_loss: 0.0050
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0048 - val_loss: 0.0050
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0048 - val_loss: 0.0049
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0049
Epoch 159/1000
32/32 [==============================] - 0s 140us/step - loss: 0.0047 - val_loss: 0.0048
Epoch 160/1000
32/32 [==============================] - 0s 188us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0028 - val_loss: 0.0026
Epoch 234/1000
32/32 [==============================] - 0s 137us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 235/1000
32/32 [==============================] - 0s 130us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 236/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 237/1000
32/32 [==============================] - 0s 152us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 0.0024
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 310/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 311/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 312/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 313/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 315/1000
32/32 [==============================] - 0s 184us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 0.0012
Epoch 317/1000
32/32 [==============================] - 0s 170us/step - loss: 0.0019 - val_loss: 0.0012
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 759/1000
32/32 [==============================] - 0s 165us/step - loss: 4.2267e-04 - val_loss: 9.0178e-06
Epoch 760/1000
32/32 [==============================] - 0s 188us/step - loss: 4.2159e-04 - val_loss: 8.9940e-06
Epoch 761/1000
32/32 [==============================] - 0s 155us/step - loss: 4.2050e-04 - val_loss: 8.9699e-06
Epoch 762/1000
32/32 [==============================] - 0s 141us/step - loss: 4.1943e-04 - val_loss: 8.9473e-06
Epoch 763/1000
32/32 [==============================] - 0s 155us/step - loss: 4.1835e-04 - val_loss: 8.9215e-06
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1728e-04 - val_loss: 8.8982e-06
Epoch 765/1000
32/32 [==============================] - 0s 148us/step - loss: 4.1621e-04 - val_loss: 8.8754e-06
Epoch 766/1000
32/32 [==============================] - 0s 187us/step - loss: 4.1515e-04 - val_loss: 8.8529e-06
Epoch 767/1000
32/32 [==============================] - 0s 155us/step - loss: 4.1409e-04 - val_loss: 8.8

Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5053e-04 - val_loss: 8.3749e-06
Epoch 834/1000
32/32 [==============================] - 0s 165us/step - loss: 3.4966e-04 - val_loss: 8.3753e-06
Epoch 835/1000
32/32 [==============================] - 0s 168us/step - loss: 3.4879e-04 - val_loss: 8.3753e-06
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4792e-04 - val_loss: 8.3766e-06
Epoch 837/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4706e-04 - val_loss: 8.3771e-06
Epoch 838/1000
32/32 [==============================] - 0s 186us/step - loss: 3.4620e-04 - val_loss: 8.3784e-06
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4533e-04 - val_loss: 8.3791e-06
Epoch 840/1000
32/32 [==============================] - 0s 166us/step - loss: 3.4448e-04 - val_loss: 8.3790e-06
Epoch 841/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4362e-04 - val_loss: 8.3

Epoch 907/1000
32/32 [==============================] - 0s 163us/step - loss: 2.9208e-04 - val_loss: 8.4439e-06
Epoch 908/1000
32/32 [==============================] - 0s 124us/step - loss: 2.9136e-04 - val_loss: 8.4449e-06
Epoch 909/1000
32/32 [==============================] - 0s 186us/step - loss: 2.9065e-04 - val_loss: 8.4441e-06
Epoch 910/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8994e-04 - val_loss: 8.4448e-06
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8924e-04 - val_loss: 8.4442e-06
Epoch 912/1000
32/32 [==============================] - 0s 155us/step - loss: 2.8853e-04 - val_loss: 8.4426e-06
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8784e-04 - val_loss: 8.4427e-06
Epoch 914/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8713e-04 - val_loss: 8.4396e-06
Epoch 915/1000
32/32 [==============================] - 0s 187us/step - loss: 2.8643e-04 - val_loss: 8.4

Epoch 981/1000
32/32 [==============================] - 0s 154us/step - loss: 2.4425e-04 - val_loss: 8.2262e-06
Epoch 982/1000
32/32 [==============================] - 0s 192us/step - loss: 2.4367e-04 - val_loss: 8.2231e-06
Epoch 983/1000
32/32 [==============================] - 0s 124us/step - loss: 2.4309e-04 - val_loss: 8.2179e-06
Epoch 984/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4251e-04 - val_loss: 8.2120e-06
Epoch 985/1000
32/32 [==============================] - 0s 187us/step - loss: 2.4192e-04 - val_loss: 8.2072e-06
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4135e-04 - val_loss: 8.2001e-06
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4077e-04 - val_loss: 8.1941e-06
Epoch 988/1000
32/32 [==============================] - 0s 155us/step - loss: 2.4020e-04 - val_loss: 8.1890e-06
Epoch 989/1000
32/32 [==============================] - 0s 124us/step - loss: 2.3962e-04 - val_loss: 8.1

[array([[-0.39856923, -0.31191564, -0.5037411 , -0.60878336, -0.36186352,
         -0.53622544, -0.71625394,  0.5010579 ,  0.2830679 ,  0.2544607 ],
        [ 0.5505844 , -0.23389895,  0.5067956 , -0.44076166, -0.65445596,
         -0.2462683 ,  0.57102466, -0.48279908, -0.6842674 , -0.03347318],
        [ 0.89711744, -1.8881333 , -0.9163159 , -0.15211566,  0.7026993 ,
         -1.0187443 , -0.6227963 ,  0.8958008 ,  0.6999281 ,  0.910984  ]],
       dtype=float32),
 array([-0.62097865, -0.31833005,  0.46635935,  0.46035165, -0.31207067,
        -0.42588407,  0.6433206 , -0.33717394,  0.1798445 ,  0.38315478],
       dtype=float32),
 array([[-8.7912518e-01,  9.1336265e-02, -6.6903424e-01, -2.5425944e-01,
          7.2408337e-03,  3.4046773e-02,  7.4167222e-01, -7.6350212e-01,
          1.9926375e-02,  6.5992242e-01],
        [-3.2849002e-01,  4.6211648e-01, -3.2892880e-01, -1.0348189e-01,
         -2.0519510e-01,  2.8622195e-01,  1.5567784e-01, -1.9389203e-01,
          3.2926512e-01, 

In [71]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure5.h5')

In [72]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_6(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_47 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_48 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0132 - val_loss: 0.0042
Epoch 72/1000
32/32 [==============================] - 0s 158us/step - loss: 0.0118 - val_loss: 0.0044
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0117 - val_loss: 0.0049
Epoch 74/1000
32/32 [==============================] - 0s 162us/step - loss: 0.0124 - val_loss: 0.0052
Epoch 75/1000
32/32 [==============================] - 0s 138us/step - loss: 0.0134 - val_loss: 0.0052
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0140 - val_loss: 0.0047
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0140 - val_loss: 0.0039
Epoch 78/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0133 - val_loss: 0.0029
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0122 - val_loss: 0.0021
Epoch 80/1000
32/32 [==============================] - 0s 132us/step - lo

Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0049 - val_loss: 6.0043e-04
Epoch 150/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0049 - val_loss: 5.8758e-04
Epoch 151/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0048 - val_loss: 5.7490e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0048 - val_loss: 5.6273e-04
Epoch 153/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0048 - val_loss: 5.5137e-04
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0047 - val_loss: 5.4108e-04
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 5.3203e-04
Epoch 156/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0047 - val_loss: 5.2419e-04
Epoch 157/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0046 - val_loss: 5.1743e-04
Epoch 158/1000
32/32 [======

Epoch 225/1000
32/32 [==============================] - 0s 219us/step - loss: 0.0030 - val_loss: 2.8933e-04
Epoch 226/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0030 - val_loss: 2.8740e-04
Epoch 227/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0030 - val_loss: 2.8549e-04
Epoch 228/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0030 - val_loss: 2.8361e-04
Epoch 229/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0029 - val_loss: 2.8180e-04
Epoch 230/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0029 - val_loss: 2.8004e-04
Epoch 231/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0029 - val_loss: 2.7831e-04
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0029 - val_loss: 2.7664e-04
Epoch 233/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0029 - val_loss: 2.7498e-04
Epoch 234/1000
32/32 [======

Epoch 301/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 1.8835e-04
Epoch 302/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 1.8739e-04
Epoch 303/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 1.8641e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 1.8545e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 1.8449e-04
Epoch 306/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 1.8353e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 1.8259e-04
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 1.8163e-04
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 1.8071e-04
Epoch 310/1000
32/32 [======

...

Epoch 748/1000
32/32 [==============================] - 0s 187us/step - loss: 1.6848e-04 - val_loss: 4.6596e-05
Epoch 749/1000
32/32 [==============================] - 0s 138us/step - loss: 1.6772e-04 - val_loss: 4.6582e-05
Epoch 750/1000
32/32 [==============================] - 0s 125us/step - loss: 1.6695e-04 - val_loss: 4.6566e-05
Epoch 751/1000
32/32 [==============================] - 0s 171us/step - loss: 1.6619e-04 - val_loss: 4.6541e-05
Epoch 752/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6543e-04 - val_loss: 4.6532e-05
Epoch 753/1000
32/32 [==============================] - 0s 143us/step - loss: 1.6468e-04 - val_loss: 4.6518e-05
Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 1.6394e-04 - val_loss: 4.6502e-05
Epoch 755/1000
32/32 [==============================] - 0s 155us/step - loss: 1.6319e-04 - val_loss: 4.6492e-05
Epoch 756/1000
32/32 [==============================] - 0s 178us/step - loss: 1.6245e-04 - val_loss: 4.6

Epoch 822/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2256e-04 - val_loss: 4.6708e-05
Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2207e-04 - val_loss: 4.6724e-05
Epoch 824/1000
32/32 [==============================] - 0s 168us/step - loss: 1.2159e-04 - val_loss: 4.6741e-05
Epoch 825/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2111e-04 - val_loss: 4.6761e-05
Epoch 826/1000
32/32 [==============================] - 0s 187us/step - loss: 1.2063e-04 - val_loss: 4.6783e-05
Epoch 827/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2015e-04 - val_loss: 4.6801e-05
Epoch 828/1000
32/32 [==============================] - 0s 183us/step - loss: 1.1968e-04 - val_loss: 4.6814e-05
Epoch 829/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1921e-04 - val_loss: 4.6831e-05
Epoch 830/1000
32/32 [==============================] - 0s 157us/step - loss: 1.1874e-04 - val_loss: 4.6

Epoch 896/1000
32/32 [==============================] - 0s 155us/step - loss: 9.3507e-05 - val_loss: 4.8726e-05
Epoch 897/1000
32/32 [==============================] - 0s 187us/step - loss: 9.3199e-05 - val_loss: 4.8757e-05
Epoch 898/1000
32/32 [==============================] - 0s 156us/step - loss: 9.2890e-05 - val_loss: 4.8791e-05
Epoch 899/1000
32/32 [==============================] - 0s 155us/step - loss: 9.2587e-05 - val_loss: 4.8829e-05
Epoch 900/1000
32/32 [==============================] - 0s 187us/step - loss: 9.2284e-05 - val_loss: 4.8866e-05
Epoch 901/1000
32/32 [==============================] - 0s 155us/step - loss: 9.1979e-05 - val_loss: 4.8905e-05
Epoch 902/1000
32/32 [==============================] - 0s 142us/step - loss: 9.1682e-05 - val_loss: 4.8932e-05
Epoch 903/1000
32/32 [==============================] - 0s 156us/step - loss: 9.1387e-05 - val_loss: 4.8973e-05
Epoch 904/1000
32/32 [==============================] - 0s 151us/step - loss: 9.1089e-05 - val_loss: 4.9

Epoch 970/1000
32/32 [==============================] - 0s 155us/step - loss: 7.5074e-05 - val_loss: 5.1512e-05
Epoch 971/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4881e-05 - val_loss: 5.1553e-05
Epoch 972/1000
32/32 [==============================] - 0s 130us/step - loss: 7.4685e-05 - val_loss: 5.1587e-05
Epoch 973/1000
32/32 [==============================] - 0s 175us/step - loss: 7.4486e-05 - val_loss: 5.1621e-05
Epoch 974/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4290e-05 - val_loss: 5.1661e-05
Epoch 975/1000
32/32 [==============================] - 0s 142us/step - loss: 7.4100e-05 - val_loss: 5.1699e-05
Epoch 976/1000
32/32 [==============================] - 0s 186us/step - loss: 7.3907e-05 - val_loss: 5.1748e-05
Epoch 977/1000
32/32 [==============================] - 0s 162us/step - loss: 7.3720e-05 - val_loss: 5.1791e-05
Epoch 978/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3530e-05 - val_loss: 5.1

[array([[ 0.28087384,  0.406483  ,  0.13724387,  0.95592755,  0.407868  ,
         -0.60154533,  0.10206072, -0.20238079, -1.1041923 ,  0.24763638],
        [-0.9669209 ,  0.03169855, -1.2415669 , -0.3425579 , -0.54937434,
         -0.3668389 , -0.42208496,  1.972765  ,  0.49491808, -0.44845665],
        [ 1.0381293 ,  0.43886545,  0.40161824,  0.26361358, -0.8454036 ,
         -1.1648798 , -0.64350915, -0.76367366, -0.23929536, -0.24984089]],
       dtype=float32),
 array([-0.18567239, -0.19960202, -0.4747588 , -0.39614034,  0.44003484,
        -0.5078565 , -0.14193599,  0.3229062 ,  0.35802522,  0.36925665],
       dtype=float32),
 array([[-0.19537196, -0.23077053, -0.05636588, -0.42498633,  0.5060052 ,
          0.3231616 , -0.02683786,  0.46024093, -0.1698411 ,  0.00524977,
         -0.2989134 , -0.51179147, -0.31882367,  0.8523579 , -0.57084537],
        [-0.51346296, -0.58529073, -0.02220993, -0.05467043, -0.31122902,
         -0.19538184, -0.2086527 , -0.09757219,  0.58176947, -

In [73]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure6.h5')